GPU-based computation of global statistics (e.g., mean intensity across all pixels).
Please update the h5_path accordingly.

In [ ]:
import h5py
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import time

# GPU可用性チェック
try:
    import cupy as cp
    GPU_AVAILABLE = True
    print("🚀 GPU処理が利用可能です")
    
    # GPU情報表示
    gpu_props = cp.cuda.runtime.getDeviceProperties(0)
    gpu_name = gpu_props['name'].decode('utf-8')
    total_memory_gb = cp.cuda.Device().mem_info[1] / (1024**3)
    print(f"🎮 GPU: {gpu_name}, VRAM: {total_memory_gb:.1f}GB")
    
except ImportError:
    GPU_AVAILABLE = False
    print("⚠️ CPU処理を使用します")

# === パラメータ ===
h5_path = r"I:\20240508-174849tdTomato-12mW-3_raw_gzip\20240508-174849tdTomato-12mW-3_raw_gzip.h5"
dataset_name = "/default"
output_csv = os.path.splitext(h5_path)[0] + "_top90_mean.csv"

# 高速化パラメータ
offset_value = 100
exclude_value = -100
chunk_size = 50 if GPU_AVAILABLE else 25  # GPU使用時はより大きなチャンク

def process_volume_gpu(vol_chunk, c, offset_value, exclude_value):
    """GPU用のボリューム統計処理"""
    try:
        # GPUに転送
        vol_gpu = cp.asarray(vol_chunk, dtype=cp.float32)
        vol_gpu -= offset_value
        
        # 各統計の計算
        stats = {}
        
        # -100除外統計
        valid_mask = vol_gpu != exclude_value
        valid_vals = vol_gpu[valid_mask]
        
        # if len(valid_vals) > 0:
        #     stats['sum'] = float(cp.sum(valid_vals))
        #     stats['mean'] = float(cp.mean(valid_vals))
        # else:
        #     stats['sum'] = 0.0
        #     stats['mean'] = 0.0
        
        # # 正の値のみの統計
        # pos_mask = vol_gpu > 0
        # pos_vals = vol_gpu[pos_mask]
        
        # if len(pos_vals) > 0:
        #     stats['possum'] = float(cp.sum(pos_vals))
        #     stats['posmean'] = float(cp.mean(pos_vals))
        # else:
        #     stats['possum'] = 0.0
        #     stats['posmean'] = 0.0
        
        # # チャンネル別閾値統計
        # if c == 0:
        #     thresh_mask = vol_gpu > 10
        #     thresh_vals = vol_gpu[thresh_mask]
        #     stats['posmean_gcamp5_td10'] = float(cp.mean(thresh_vals)) if len(thresh_vals) > 0 else 0.0
        # elif c == 1:
        #     thresh_mask = vol_gpu > 5
        #     thresh_vals = vol_gpu[thresh_mask]
        #     stats['posmean_gcamp5_td10'] = float(cp.mean(thresh_vals)) if len(thresh_vals) > 0 else 0.0
        # else:
        #     stats['posmean_gcamp5_td10'] = 0.0
        
        # 上位10%, 50%, 90%の平均値
        # percentiles = [90, 50, 10]
        percentiles = [90]
        for p in percentiles:
            if len(valid_vals) > 0:
                thresh = cp.percentile(valid_vals, p)
                top_vals = valid_vals[valid_vals >= thresh]
                stats[f'top{p}_mean'] = float(cp.mean(top_vals)) if len(top_vals) > 0 else 0.0
            else:
                stats[f'top{p}_mean'] = 0.0
        
        
        # メモリクリーンアップ
        del vol_gpu, valid_vals, top_vals, thresh
        if 'thresh_vals' in locals():
            del thresh_vals
        
        return stats
        
    except Exception as e:
        print(f"⚠️ GPU処理エラー: {e}")
        # CPUフォールバック
        return process_volume_cpu(vol_chunk, c, offset_value, exclude_value)

def process_volume_cpu(vol_chunk, c, offset_value, exclude_value):
    """CPU用のボリューム統計処理（ベクトル化最適化）"""
    vol = vol_chunk.astype(np.float32)
    vol -= offset_value
    
    stats = {}
    
    # -100除外統計（ベクトル化）
    valid_mask = vol != exclude_value
    if np.any(valid_mask):
        valid_vals = vol[valid_mask]
        stats['sum'] = float(np.sum(valid_vals))
        stats['mean'] = float(np.mean(valid_vals))
    else:
        stats['sum'] = 0.0
        stats['mean'] = 0.0
    
    # 正の値のみの統計（ベクトル化）
    pos_mask = vol > 0
    if np.any(pos_mask):
        pos_vals = vol[pos_mask]
        stats['possum'] = float(np.sum(pos_vals))
        stats['posmean'] = float(np.mean(pos_vals))
    else:
        stats['possum'] = 0.0
        stats['posmean'] = 0.0
    
    # チャンネル別閾値統計
    if c == 0:
        thresh_mask = vol > 10
        stats['posmean_gcamp5_td10'] = float(np.mean(vol[thresh_mask])) if np.any(thresh_mask) else 0.0
    elif c == 1:
        thresh_mask = vol > 5
        stats['posmean_gcamp5_td10'] = float(np.mean(vol[thresh_mask])) if np.any(thresh_mask) else 0.0
    else:
        stats['posmean_gcamp5_td10'] = 0.0
    
    return stats

def process_statistics_optimized():
    """最適化された統計処理（チャンク+GPU/CPU）"""
    start_time = time.time()
    
    with h5py.File(h5_path, "r") as f:
        dset = f[dataset_name]
        t_len, c_len, z_len, y_len, x_len = dset.shape
        print(f"📐 Dataset shape: {dset.shape}")
        print(f"🚀 処理モード: {'GPU' if GPU_AVAILABLE else 'CPU'}")
        print(f"📦 チャンクサイズ: {chunk_size}")
        
        # 結果辞書の初期化
        result = {"frame": list(range(t_len))}
        for c in range(c_len):
            # result[f"ch{c}_sum"] = []
            # result[f"ch{c}_mean"] = []
            # result[f"ch{c}_possum"] = []
            # result[f"ch{c}_posmean"] = []
            # result[f"ch{c}_posmean_gcamp5_td10"] = []
            result[f"ch{c}_top90_mean"] = []
            # result[f"ch{c}_top50_mean"] = []
            # result[f"ch{c}_top10_mean"] = []
        
        # チャンク処理でメモリ効率化
        for t_start in tqdm(range(0, t_len, chunk_size), desc="🎯 統計計算処理"):
            t_end = min(t_start + chunk_size, t_len)
            
            # 複数フレームを一括読み込み
            chunk_data = dset[t_start:t_end]  # shape: (chunk_size, c, z, y, x)
            
            for i, t in enumerate(range(t_start, t_end)):
                for c in range(c_len):
                    vol_chunk = chunk_data[i, c]  # shape: (z, y, x)
                    
                    # GPU/CPU処理の選択
                    if GPU_AVAILABLE:
                        stats = process_volume_gpu(vol_chunk, c, offset_value, exclude_value)
                    else:
                        stats = process_volume_cpu(vol_chunk, c, offset_value, exclude_value)
                    
                    # 結果の蓄積
                    # result[f"ch{c}_sum"].append(stats['sum'])
                    # result[f"ch{c}_mean"].append(stats['mean'])
                    # result[f"ch{c}_possum"].append(stats['possum'])
                    # result[f"ch{c}_posmean"].append(stats['posmean'])
                    # result[f"ch{c}_posmean_gcamp5_td10"].append(stats['posmean_gcamp5_td10'])
                    result[f"ch{c}_top90_mean"].append(stats['top90_mean'])
                    # result[f"ch{c}_top50_mean"].append(stats['top50_mean'])
                    # result[f"ch{c}_top10_mean"].append(stats['top10_mean'])
                    
                    
                    
            
            # 進捗情報表示
            if t_start % (chunk_size * 10) == 0:
                elapsed = time.time() - start_time
                processed = t_end
                fps = processed / elapsed if elapsed > 0 else 0
                remaining = t_len - processed
                eta_minutes = (remaining / fps / 60) if fps > 0 else 0
                
                print(f"  📊 進捗: {processed}/{t_len} ({100*processed/t_len:.1f}%)")
                print(f"  🚀 速度: {fps:.1f}フレーム/秒, ETA: {eta_minutes:.1f}分")
            
            # GPUメモリクリーンアップ
            if GPU_AVAILABLE:
                try:
                    cp.get_default_memory_pool().free_all_blocks()
                except:
                    pass
    
    # 処理結果サマリー
    total_elapsed = time.time() - start_time
    average_fps = t_len / total_elapsed if total_elapsed > 0 else 0
    
    print(f"\n🎉 統計計算完了!")
    print(f"📊 処理フレーム数: {t_len}")
    print(f"🔄 処理チャンネル数: {c_len}")
    print(f"⏱️ 総処理時間: {total_elapsed:.1f}秒 ({total_elapsed/60:.1f}分)")
    print(f"🚀 平均処理速度: {average_fps:.1f}フレーム/秒")
    
    return result

# === 最適化処理実行 ===
print("🎯 最適化された統計計算を開始...")
result = process_statistics_optimized()

# === 保存処理 ===
print("💾 CSV保存中...")
df = pd.DataFrame(result)

try:
    df.to_csv(output_csv, index=False)
    print(f"✅ 最適化統計処理完了: {output_csv}")
    print(f"📈 出力データ形状: {df.shape}")
    
    # データサンプル表示
    print("\n📊 データサンプル:")
    print(df.head())
    
except PermissionError:
    print("❌ 書き込みに失敗しました。ファイルが開かれているか、書き込み権限がありません:")
    print("🔒 ファイルを閉じているか、別の保存先を指定してください。")
    print("📄 試行した出力先:", output_csv)
    
    # 代替保存先の提案
    alternative_path = output_csv.replace(".csv", f"_backup_{int(time.time())}.csv")
    try:
        df.to_csv(alternative_path, index=False)
        print(f"✅ 代替パスに保存完了: {alternative_path}")
    except Exception as e:
        print(f"❌ 代替保存も失敗: {e}")

# === メモリクリーンアップ ===
try:
    del result, df
    if GPU_AVAILABLE:
        cp.get_default_memory_pool().free_all_blocks()
    print("🧹 メモリクリーンアップ完了")
except:
    pass

Accelerated fading correction processing (GPU parallel processing) with support for 2 channels. Please modify csv_path, h5_input_path, and h5_output_path as needed.

In [ ]:
import h5py
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from tqdm import tqdm
import warnings
import time
import cupy as cp
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt

# === パラメータ ===
calc_method = "_top90_mean"  # 使用する列のメソッド（top50_mean, top90_meanなど）
csv_path = r"I:\20240508-174849tdTomato-12mW-3_raw_gzip\20240508-174849tdTomato-12mW-3_raw_gzip_top90_mean.csv"
h5_input_path = r"I:\20240508-174849tdTomato-12mW-3_raw_gzip\20240508-174849tdTomato-12mW-3_raw_gzip.h5"
h5_output_path = fr"I:\20240508-174849tdTomato-12mW-3_raw_gzip\20240508-174849tdTomato-12mW-3_raw_gzip_bleachcorrect_{calc_method}.h5"
offset_value = 100
scale_margin = 1.2
chunk_size = 60  # メモリーサイズに応じて設定

# === 褪色関数定義 ===
def double_exp(t, a, b, c, d):
    return a * np.exp(-b * t) + c * np.exp(-d * t)

# === チャンネル別褪色カーブ推定 ===
def estimate_bleach_curves():
    """各チャンネルの褪色カーブを独立して推定"""
    df = pd.read_csv(csv_path)
    t = np.arange(len(df))
    
    
    bleach_params = {}
    scale_factors = {}
    
    # チャンネル数の自動検出
    mean_columns = [col for col in df.columns if col.endswith(calc_method)]
    channel_numbers = [int(col.split('_')[0][2:]) for col in mean_columns if col.startswith('ch')]
    
    print(f"🔍 検出されたチャンネル: {channel_numbers}")
    
    for ch_num in channel_numbers:
        column_name = f"ch{ch_num}{calc_method}"
        
        if column_name not in df.columns:
            print(f"⚠️ 列 '{column_name}' が見つかりません。スキップします。")
            continue
            
        print(f"\n--- Channel {ch_num} 褪色カーブ推定 ---")
        
        y_raw = df[column_name].to_numpy()
        # SGフィルタ適用（ウィンドウサイズ13, 多項式次数1）
        y_smooth = savgol_filter(y_raw, window_length=13, polyorder=1, mode='interp')
        
        # 初期値と上限の推定
        a0 = c0 = y_smooth[0] / 2
        p0 = [a0, 0.01, c0, 0.001]
        y_max = np.nanmax(y_smooth)
        upper_limit = y_max * scale_margin
        bounds = ([0, 0, 0, 0], [upper_limit, 1, upper_limit, 1])
        
        try:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                popt, _ = curve_fit(double_exp, t, y_smooth, p0=p0, bounds=bounds, maxfev=10000)
            
            fitted = double_exp(t, *popt)
            scale = double_exp(0, *popt) / fitted  # t=0で正規化
            
            bleach_params[ch_num] = popt
            scale_factors[ch_num] = scale
      

            # 横並びで全チャンネル分のプロットを作成・保存
            fig, axes = plt.subplots(1, len(channel_numbers), figsize=(6 * len(channel_numbers), 4), sharey=True)
            if len(channel_numbers) == 1:
                axes = [axes]
            for idx, ch in enumerate(channel_numbers):
                col_name = f"ch{ch}{calc_method}"
                y = df[col_name].to_numpy()
                if ch in bleach_params:
                    fit = double_exp(t, *bleach_params[ch])
                    # 褪色補正後データ
                    corrected = y * scale_factors[ch]
                else:
                    fit = np.ones_like(t) * y[0]
                    corrected = y
                axes[idx].plot(t, y, 'o', label=f'Ch{ch} Raw')
                axes[idx].plot(t, fit, '-', label=f'Ch{ch} Fit')
                axes[idx].plot(t, corrected, '--', label=f'Ch{ch} Corrected')
                axes[idx].set_title(f'Channel {ch} Bleach Curve')
                axes[idx].set_xlabel('Time')
                axes[idx].legend()
            axes[0].set_ylabel('Intensity')
            plt.tight_layout()
            plt.savefig(f'bleach_curve_all_channels{calc_method}.png')
            plt.show()
            
            print(f"✅ Ch{ch_num} フィッティング完了: a={popt[0]:.2f}, b={popt[1]:.4f}, c={popt[2]:.2f}, d={popt[3]:.4f}")
            
        except Exception as e:
            print(f"❌ Ch{ch_num} フィッティング失敗: {e}")
            # フォールバック: スケールファクター=1（補正なし）
            scale_factors[ch_num] = np.ones_like(t)
    
    return scale_factors

# 褪色カーブ推定実行
scale_factors_all = estimate_bleach_curves()
print("\n✅ 全チャンネルの褪色カーブ推定完了")

# GPU版（チャンネル別補正対応）
try:
    GPU_AVAILABLE = True
    print("🚀 GPU処理が利用可能です")
    
    # GPU情報表示
    gpu_props = cp.cuda.runtime.getDeviceProperties(0)
    gpu_name = gpu_props['name'].decode('utf-8')
    total_memory_gb = cp.cuda.Device().mem_info[1] / (1024**3)
    print(f"🎮 GPU: {gpu_name}")
    print(f"💾 VRAM: {total_memory_gb:.1f}GB")
    
except ImportError:
    GPU_AVAILABLE = False
    print("⚠️ CuPyが利用できません。CPU処理を使用します")

def process_with_gpu_multichannel():
    """チャンネル別褪色補正GPU処理（全ボリューム対応）"""
    if not GPU_AVAILABLE:
        print("❌ GPU処理をスキップします")
        return
    
    start_time = time.time()
    
    with h5py.File(h5_input_path, "r") as f_in, h5py.File(h5_output_path, "w") as f_out:
        dset_in = f_in["/default"]
        T_full, C, Z, Y, X = dset_in.shape
        
        print(f"📐 データ形状: (T={T_full}, C={C}, Z={Z}, Y={Y}, X={X})")
        print(f"🎯 処理対象: 全ボリューム（{T_full}フレーム）")
        
        # 出力データセット作成（全ボリューム）
        dset_out = f_out.create_dataset(
            "/default", 
            shape=(T_full, C, Z, Y, X),
            dtype='uint16',
            chunks=(min(chunk_size//4, 20), 1, Z, Y, X),
            compression="gzip",
            compression_opts=1
        )
        
        # メタデータ追加
        dset_out.attrs['processing_mode'] = 'multichannel_bleach_correction_full'
        dset_out.attrs['chunk_size'] = chunk_size
        dset_out.attrs['offset_value'] = offset_value
        dset_out.attrs['processed_frames'] = T_full
        dset_out.attrs['frame_range'] = f"0-{T_full-1}"
        
        # チャンネル別スケールファクターの準備
        scale_factors_gpu = {}
        for ch_num in range(C):
            if ch_num in scale_factors_all:
                # 全フレーム分を取得
                scale_factors_gpu[ch_num] = cp.asarray(
                    scale_factors_all[ch_num], dtype=cp.float32
                )
                print(f"✅ Ch{ch_num}: GPU用スケールファクター準備完了（{T_full}フレーム）")
            else:
                scale_factors_gpu[ch_num] = cp.ones(T_full, dtype=cp.float32)
                print(f"⚠️ Ch{ch_num}: スケールファクターなし（補正スキップ）")
        
        # チャンク処理メインループ（全フレーム）
        for t_start in tqdm(range(0, T_full, chunk_size), desc="🎮 GPU多チャンネル褪色補正（全ボリューム）"):
            t_end = min(t_start + chunk_size, T_full)
            current_chunk_size = t_end - t_start
            
            try:
                # メモリ使用量監視
                memory_before = cp.cuda.Device().mem_info[0] / (1024**3)
                
                # チャンネル別処理
                for c in range(C):
                    # CPUからGPUへデータ転送
                    vol_chunk = cp.asarray(
                        dset_in[t_start:t_end, c, :, :, :], 
                        dtype=cp.float32
                    )
                    
                    # チャンネル専用スケールファクター取得
                    scale_chunk = scale_factors_gpu[c][t_start:t_end]
                    
                    # GPU上でベクトル化処理
                    vol_chunk -= offset_value  # オフセット減算
                    vol_chunk *= scale_chunk.reshape(-1, 1, 1, 1)  # チャンネル別褪色補正
                    vol_chunk = cp.clip(vol_chunk, 0, 65535)  # クリッピング
                    
                    # 型変換と結果保存
                    vol_chunk = vol_chunk.astype(cp.uint16)
                    result = cp.asnumpy(vol_chunk)
                    dset_out[t_start:t_end, c, :, :, :] = result
                    
                    # メモリクリーンアップ（チャンネル毎）
                    del vol_chunk, result
                    cp.cuda.Stream.null.synchronize()
                
                # 進捗情報表示
                if t_start % (chunk_size * 5) == 0:
                    memory_after = cp.cuda.Device().mem_info[0] / (1024**3)
                    elapsed = time.time() - start_time
                    processed_frames = t_end
                    fps = processed_frames / elapsed if elapsed > 0 else 0
                    remaining_frames = T_full - processed_frames
                    eta_minutes = (remaining_frames / fps / 60) if fps > 0 else 0
                    
                    print(f"  📊 進捗: {processed_frames}/{T_full} ({100*processed_frames/T_full:.1f}%)")
                    print(f"  🚀 速度: {fps:.1f}fps, VRAM: {memory_before:.1f}GB, ETA: {eta_minutes:.1f}分")
                    
            except cp.cuda.memory.OutOfMemoryError:
                print(f"💥 GPU メモリ不足 (chunk_size={chunk_size})")
                cp.get_default_memory_pool().free_all_blocks()
                continue
                
            except Exception as e:
                print(f"❌ 処理エラー (t={t_start}-{t_end}): {e}")
                continue
    
    # 最終メモリクリーンアップ
    try:
        # 辞書のキーのリストを事前に取得してからイテレート
        channel_keys = list(scale_factors_gpu.keys())
        for ch_num in channel_keys:
            if ch_num in scale_factors_gpu:
                del scale_factors_gpu[ch_num]
        
        # GPU メモリプールのクリーンアップ
        cp.get_default_memory_pool().free_all_blocks()
        
    except Exception as cleanup_error:
        print(f"⚠️ メモリクリーンアップ中にエラー: {cleanup_error}")
        # 強制的にメモリプールをクリア
        try:
            cp.get_default_memory_pool().free_all_blocks()
        except:
            pass
    
    # 処理結果サマリー
    total_elapsed = time.time() - start_time
    average_fps = T_full / total_elapsed if total_elapsed > 0 else 0
    
    print(f"\n🎉 全ボリューム多チャンネル褪色補正GPU処理完了!")
    print(f"📊 処理フレーム数: {T_full}")
    print(f"🔄 処理チャンネル数: {C}")
    print(f"⏱️  総処理時間: {total_elapsed/60:.1f}分")
    print(f"🚀 平均処理速度: {average_fps:.1f}フレーム/秒")
    print(f"💾 出力ファイル: {h5_output_path}")

# === 実行 ===
if GPU_AVAILABLE and scale_factors_all:
    print("🎯 チャンネル別褪色補正GPU処理を開始します（全ボリューム）...")
    process_with_gpu_multichannel()
else:
    print("⚠️ GPU処理またはスケールファクターが利用できません")
    print(f"GPU利用可能: {GPU_AVAILABLE}")
    print(f"スケールファクター: {list(scale_factors_all.keys()) if scale_factors_all else 'なし'}")

Orthogonal View tiff from hdf5

In [ ]:
import h5py
import numpy as np
import tifffile as tiff
import os
from tqdm import tqdm

# GPU可用性チェック
try:
    import cupy as cp
    GPU_AVAILABLE = True
    print("🚀 GPU処理が利用可能です")
    
    # GPU情報表示
    gpu_props = cp.cuda.runtime.getDeviceProperties(0)
    gpu_name = gpu_props['name'].decode('utf-8')
    total_memory_gb = cp.cuda.Device().mem_info[1] / (1024**3)
    print(f"🎮 GPU: {gpu_name}")
    print(f"💾 VRAM: {total_memory_gb:.1f}GB")
    
except ImportError:
    GPU_AVAILABLE = False
    print("⚠️ CPU処理を使用します")

# === パラメータ ===
hdf5_file = h5_output_path
output_path = r"I:\20240508-174849tdTomato-12mW-3_raw_gzip\bleachcorrect"
dirname = os.path.splitext(os.path.basename(hdf5_file))[0]
chunk_size = 100 if GPU_AVAILABLE else 50  # GPU使用時はより大きなチャンク

if not os.path.exists(output_path):
    os.makedirs(output_path)

# === HDF5ファイル情報取得 ===
with h5py.File(hdf5_file, 'r') as file:
    array = file['default']
    total_volumes, channels, z, y, x = array.shape
    print(f"📐 データ形状: (T={total_volumes}, C={channels}, Z={z}, Y={y}, X={x})")
    print(f"🎯 処理対象: 全ボリューム（{total_volumes}フレーム）")

# === 出力配列の準備（全ボリューム） ===
orthogonal_view = np.zeros((total_volumes, channels, y + z + 3, x + z + 3), dtype='uint16')

def process_cpu_chunk(chunk_data, t_start, t_end):
    """CPU処理用のヘルパー関数"""
    for i, t in enumerate(range(t_start, t_end)):
        for c in range(channels):
            data = chunk_data[i, c]  # (Z, Y, X)
            
            # CPU上で最大投影
            xy_proj = np.max(data, axis=0)  # (Y, X)
            yz_proj = np.max(data, axis=2).T  # (Z, Y)
            xz_proj = np.max(data, axis=1)  # (Z, X)
            
            # 直交ビューへの配置
            orthogonal_view[t, c, 0:y, 0:x] = xy_proj
            orthogonal_view[t, c, 0:y, x+3:x+z+3] = yz_proj
            orthogonal_view[t, c, y+3:y+z+3, 0:x] = xz_proj
            orthogonal_view[t, c, y+z+2, x+z+2] = 1000

def process_orthogonal_projections():
    """直交投影処理（CPU/GPU自動選択・全ボリューム）"""
    use_gpu = GPU_AVAILABLE  # ローカル変数として処理状態を管理
    
    with h5py.File(hdf5_file, 'r') as file:
        array = file['default']
        
        for t_start in tqdm(range(0, total_volumes, chunk_size), 
                           desc="🎮 GPU直交投影（全ボリューム）" if use_gpu else "🖥️ CPU直交投影（全ボリューム）"):
            t_end = min(t_start + chunk_size, total_volumes)
            
            # チャンクデータ読み込み
            chunk_data = array[t_start:t_end]  # (chunk_size, C, Z, Y, X)
            
            if use_gpu:
                # GPU処理を試行
                try:
                    chunk_gpu = cp.asarray(chunk_data)
                    
                    for i, t in enumerate(range(t_start, t_end)):
                        for c in range(channels):
                            data_gpu = chunk_gpu[i, c]  # (Z, Y, X)
                            
                            # GPU上で最大投影
                            xy_proj = cp.max(data_gpu, axis=0)  # (Y, X)
                            yz_proj = cp.max(data_gpu, axis=2).T  # (Z, Y)
                            xz_proj = cp.max(data_gpu, axis=1)  # (Z, X)
                            
                            # CPU側に結果をコピー
                            orthogonal_view[t, c, 0:y, 0:x] = cp.asnumpy(xy_proj)
                            orthogonal_view[t, c, 0:y, x+3:x+z+3] = cp.asnumpy(yz_proj)
                            orthogonal_view[t, c, y+3:y+z+3, 0:x] = cp.asnumpy(xz_proj)
                            orthogonal_view[t, c, y+z+2, x+z+2] = 1000
                    
                    # GPUメモリクリーンアップ
                    del chunk_gpu
                    cp.get_default_memory_pool().free_all_blocks()
                    
                    # 進捗情報表示（GPU処理）
                    if t_start % (chunk_size * 5) == 0:
                        processed = t_end
                        remaining = total_volumes - processed
                        progress_pct = 100 * processed / total_volumes
                        print(f"  📊 GPU進捗: {processed}/{total_volumes} ({progress_pct:.1f}%)")
                    
                except Exception as e:
                    print(f"⚠️ GPU処理エラー、CPUにフォールバック: {e}")
                    use_gpu = False  # 今後のチャンクはCPU処理
                    # CPU処理で再試行
                    process_cpu_chunk(chunk_data, t_start, t_end)
                    
            else:
                # CPU処理
                process_cpu_chunk(chunk_data, t_start, t_end)
                
                # 進捗情報表示（CPU処理）
                if t_start % (chunk_size * 2) == 0:
                    processed = t_end
                    progress_pct = 100 * processed / total_volumes
                    print(f"  📊 CPU進捗: {processed}/{total_volumes} ({progress_pct:.1f}%)")

# === 処理実行 ===
print(f"🎯 全ボリューム（{total_volumes}フレーム）の直交投影処理を開始...")
process_orthogonal_projections()
print("🎉 直交投影処理完了!")

# === TIFF保存（全ボリューム） ===
file_name = f"orthogonal_view_{dirname}_full.tif"
output_path2 = os.path.join(output_path, file_name)

print("💾 TIFF保存中...")
print(f"📁 保存先: {output_path2}")
print(f"📊 データサイズ: {orthogonal_view.nbytes / (1024**3):.2f}GB")

try:
    tiff.imwrite(
        output_path2, 
        orthogonal_view, 
        imagej=True, 
        metadata={'axes': 'TCYX'}, 
        compression='zlib'
    )
    
    print(f"✅ 全ボリューム処理完了: {output_path2}")
    print(f"📈 出力形状: {orthogonal_view.shape}")
    print(f"📊 処理統計: {total_volumes}/{total_volumes}ボリューム (100%)")
    
except Exception as save_error:
    print(f"❌ TIFF保存エラー: {save_error}")
    print("💡 メモリ不足の可能性があります。チャンク保存を試します...")
    
    # === フォールバック: チャンク保存 ===
    save_chunk_size = 500  # 500フレームずつ保存
    
    for save_start in tqdm(range(0, total_volumes, save_chunk_size), desc="📦 チャンク保存"):
        save_end = min(save_start + save_chunk_size, total_volumes)
        chunk_name = f"orthogonal_view_{dirname}_part{save_start:05d}-{save_end:05d}.tif"
        chunk_path = os.path.join(output_path, chunk_name)
        
        chunk_data = orthogonal_view[save_start:save_end]
        tiff.imwrite(
            chunk_path,
            chunk_data,
            imagej=True,
            metadata={'axes': 'TCYX'},
            compression='zlib'
        )
        print(f"✅ 保存完了: {chunk_name}")
    
    print(f"📦 チャンク保存完了: {output_path}")

# === メモリクリーンアップ ===
try:
    del orthogonal_view
    if GPU_AVAILABLE:
        cp.get_default_memory_pool().free_all_blocks()
    print("🧹 メモリクリーンアップ完了")
except:
    pass

Bleach correction processing for multiple files

In [ ]:
##############################################################################
# 上位90%平均値計算と保存
##############################################################################
import h5py
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import time

# GPU可用性チェック
try:
    import cupy as cp
    GPU_AVAILABLE = True
    print("🚀 GPU処理が利用可能です")
    
    # GPU情報表示
    gpu_props = cp.cuda.runtime.getDeviceProperties(0)
    gpu_name = gpu_props['name'].decode('utf-8')
    total_memory_gb = cp.cuda.Device().mem_info[1] / (1024**3)
    print(f"🎮 GPU: {gpu_name}, VRAM: {total_memory_gb:.1f}GB")
    
except ImportError:
    GPU_AVAILABLE = False
    print("⚠️ CPU処理を使用します")

# === パラメータ（複数ファイル対応） ===
h5_files = [
    r"I:\20240508-174849tdTomato-12mW-3_raw_gzip\20240508-174849tdTomato-12mW-3_corrected_size_gpu.h5"
]

dataset_name = "/default"

# 高速化パラメータ
offset_value = 100
exclude_value = -100
chunk_size = 50 if GPU_AVAILABLE else 25  # GPU使用時はより大きなチャンク

def process_volume_gpu(vol_chunk, c, offset_value, exclude_value):
    """GPU用のボリューム統計処理"""
    try:
        # GPUに転送
        vol_gpu = cp.asarray(vol_chunk, dtype=cp.float32)
        vol_gpu -= offset_value
        
        # 各統計の計算
        stats = {}
        
        # -100除外統計
        valid_mask = vol_gpu != exclude_value
        valid_vals = vol_gpu[valid_mask]
        
        # 上位90%の平均値
        percentiles = [90]
        for p in percentiles:
            if len(valid_vals) > 0:
                thresh = cp.percentile(valid_vals, p)
                top_vals = valid_vals[valid_vals >= thresh]
                stats[f'top{p}_mean'] = float(cp.mean(top_vals)) if len(top_vals) > 0 else 0.0
            else:
                stats[f'top{p}_mean'] = 0.0
        
        # メモリクリーンアップ
        del vol_gpu, valid_vals, top_vals, thresh
        
        return stats
        
    except Exception as e:
        print(f"⚠️ GPU処理エラー: {e}")
        # CPUフォールバック
        return process_volume_cpu(vol_chunk, c, offset_value, exclude_value)

def process_volume_cpu(vol_chunk, c, offset_value, exclude_value):
    """CPU用のボリューム統計処理（ベクトル化最適化）"""
    vol = vol_chunk.astype(np.float32)
    vol -= offset_value
    
    stats = {}
    
    # -100除外統計（ベクトル化）
    valid_mask = vol != exclude_value
    if np.any(valid_mask):
        valid_vals = vol[valid_mask]
        
        # 上位90%の平均値
        percentiles = [90]
        for p in percentiles:
            thresh = np.percentile(valid_vals, p)
            top_vals = valid_vals[valid_vals >= thresh]
            stats[f'top{p}_mean'] = float(np.mean(top_vals)) if len(top_vals) > 0 else 0.0
    else:
        stats['top90_mean'] = 0.0
    
    return stats

def process_single_file(h5_path):
    """単一ファイルの統計処理"""
    print(f"\n🎯 処理開始: {os.path.basename(h5_path)}")
    
    output_csv = os.path.splitext(h5_path)[0] + "_top90_mean.csv"
    
    start_time = time.time()
    
    with h5py.File(h5_path, "r") as f:
        dset = f[dataset_name]
        t_len, c_len, z_len, y_len, x_len = dset.shape
        print(f"📐 Dataset shape: {dset.shape}")
        print(f"🚀 処理モード: {'GPU' if GPU_AVAILABLE else 'CPU'}")
        print(f"📦 チャンクサイズ: {chunk_size}")
        
        # 結果辞書の初期化
        result = {"frame": list(range(t_len))}
        for c in range(c_len):
            result[f"ch{c}_top90_mean"] = []
        
        # チャンク処理でメモリ効率化
        for t_start in tqdm(range(0, t_len, chunk_size), desc="🎯 統計計算処理"):
            t_end = min(t_start + chunk_size, t_len)
            
            # 複数フレームを一括読み込み
            chunk_data = dset[t_start:t_end]  # shape: (chunk_size, c, z, y, x)
            
            for i, t in enumerate(range(t_start, t_end)):
                for c in range(c_len):
                    vol_chunk = chunk_data[i, c]  # shape: (z, y, x)
                    
                    # GPU/CPU処理の選択
                    if GPU_AVAILABLE:
                        stats = process_volume_gpu(vol_chunk, c, offset_value, exclude_value)
                    else:
                        stats = process_volume_cpu(vol_chunk, c, offset_value, exclude_value)
                    
                    # 結果の蓄積
                    result[f"ch{c}_top90_mean"].append(stats['top90_mean'])
            
            # 進捗情報表示
            if t_start % (chunk_size * 10) == 0:
                elapsed = time.time() - start_time
                processed = t_end
                fps = processed / elapsed if elapsed > 0 else 0
                remaining = t_len - processed
                eta_minutes = (remaining / fps / 60) if fps > 0 else 0
                
                print(f"  📊 進捗: {processed}/{t_len} ({100*processed/t_len:.1f}%)")
                print(f"  🚀 速度: {fps:.1f}フレーム/秒, ETA: {eta_minutes:.1f}分")
            
            # GPUメモリクリーンアップ
            if GPU_AVAILABLE:
                try:
                    cp.get_default_memory_pool().free_all_blocks()
                except:
                    pass
    
    # 処理結果サマリー
    total_elapsed = time.time() - start_time
    average_fps = t_len / total_elapsed if total_elapsed > 0 else 0
    
    print(f"\n🎉 統計計算完了!")
    print(f"📊 処理フレーム数: {t_len}")
    print(f"🔄 処理チャンネル数: {c_len}")
    print(f"⏱️ 総処理時間: {total_elapsed:.1f}秒 ({total_elapsed/60:.1f}分)")
    print(f"🚀 平均処理速度: {average_fps:.1f}フレーム/秒")
    
    return result, output_csv

def save_results(result, output_csv):
    """結果の保存処理"""
    print("💾 CSV保存中...")
    df = pd.DataFrame(result)

    try:
        df.to_csv(output_csv, index=False)
        print(f"✅ 最適化統計処理完了: {output_csv}")
        print(f"📈 出力データ形状: {df.shape}")
        
        # データサンプル表示
        print("\n📊 データサンプル:")
        print(df.head())
        
    except PermissionError:
        print("❌ 書き込みに失敗しました。ファイルが開かれているか、書き込み権限がありません:")
        print("🔒 ファイルを閉じているか、別の保存先を指定してください。")
        print("📄 試行した出力先:", output_csv)
        
        # 代替保存先の提案
        alternative_path = output_csv.replace(".csv", f"_backup_{int(time.time())}.csv")
        try:
            df.to_csv(alternative_path, index=False)
            print(f"✅ 代替パスに保存完了: {alternative_path}")
        except Exception as e:
            print(f"❌ 代替保存も失敗: {e}")

# === 複数ファイル処理実行 ===
print("🎯 複数ファイルの最適化された統計計算を開始...")
print(f"📁 処理対象ファイル数: {len(h5_files)}")

total_start_time = time.time()
processed_files = 0
failed_files = []

for file_idx, h5_path in enumerate(h5_files, 1):
    print(f"\n{'='*80}")
    print(f"📂 ファイル {file_idx}/{len(h5_files)}: {os.path.basename(h5_path)}")
    print(f"{'='*80}")
    
    # ファイル存在確認
    if not os.path.exists(h5_path):
        print(f"❌ ファイルが見つかりません: {h5_path}")
        failed_files.append(h5_path)
        continue
    
    try:
        # 単一ファイル処理
        result, output_csv = process_single_file(h5_path)
        
        # 結果保存
        save_results(result, output_csv)
        
        processed_files += 1
        
        # メモリクリーンアップ
        del result
        if GPU_AVAILABLE:
            cp.get_default_memory_pool().free_all_blocks()
        
        print(f"✅ ファイル {file_idx} 完了: {os.path.basename(h5_path)}")
        
    except Exception as e:
        print(f"❌ ファイル {file_idx} 処理エラー: {e}")
        failed_files.append(h5_path)
        continue

# === 全体処理結果サマリー ===
total_elapsed = time.time() - total_start_time

print(f"\n{'='*80}")
print(f"🎉 全ファイル処理完了!")
print(f"{'='*80}")
print(f"📊 処理統計:")
print(f"  ✅ 成功: {processed_files}/{len(h5_files)} ファイル")
print(f"  ❌ 失敗: {len(failed_files)} ファイル")
print(f"⏱️  総処理時間: {total_elapsed/60:.1f}分")

if failed_files:
    print(f"\n⚠️ 失敗したファイル:")
    for failed_file in failed_files:
        print(f"  • {failed_file}")

print("\n🧹 最終メモリクリーンアップ...")
if GPU_AVAILABLE:
    try:
        cp.get_default_memory_pool().free_all_blocks()
        print("🧹 GPU メモリクリーンアップ完了")
    except:
        pass

print("🎯 全処理完了!")


########################################################################
# 褪色補正処理
########################################################################
import h5py
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from tqdm import tqdm
import warnings
import time
import cupy as cp
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
import os

# === パラメータ（複数ファイル対応） ===
calc_method = "_top90_mean"  # 使用する列のメソッド

# 処理対象ファイルの定義
file_configs = [
    {
        "name": "20240508-174849tdTomato-12mW-3",
        "csv_path": r"I:\20240508-174849tdTomato-12mW-3_raw_gzip\20240508-174849tdTomato-12mW-3_corrected_size_gpu_top90_mean.csv",
        "h5_input_path": r"I:\20240508-174849tdTomato-12mW-3_raw_gzip\20240508-174849tdTomato-12mW-3_corrected_size_gpu.h5",
        "h5_output_path": r"I:\20240508-174849tdTomato-12mW-3_raw_gzip\20240508-174849tdTomato-12mW-3_corrected_size_gpu_bleachcorrect{}.h5"
    }
]


# 共通パラメータ
offset_value = 100
scale_margin = 1.2
chunk_size = 60  # メモリーサイズに応じて設定

# === 褪色関数定義 ===
def double_exp(t, a, b, c, d):
    return a * np.exp(-b * t) + c * np.exp(-d * t)

# === チャンネル別褪色カーブ推定 ===
def estimate_bleach_curves(csv_path, file_name):
    """各チャンネルの褪色カーブを独立して推定"""
    print(f"\n🔬 {file_name} の褪色カーブ推定開始...")
    
    # ファイル存在確認
    if not os.path.exists(csv_path):
        print(f"❌ CSVファイルが見つかりません: {csv_path}")
        return None
    
    df = pd.read_csv(csv_path)
    t = np.arange(len(df))
    
    bleach_params = {}
    scale_factors = {}
    
    # チャンネル数の自動検出
    mean_columns = [col for col in df.columns if col.endswith(calc_method)]
    channel_numbers = [int(col.split('_')[0][2:]) for col in mean_columns if col.startswith('ch')]
    
    print(f"🔍 検出されたチャンネル: {channel_numbers}")
    
    if not channel_numbers:
        print(f"⚠️ 有効なチャンネルが見つかりません")
        return None
    
    for ch_num in channel_numbers:
        column_name = f"ch{ch_num}{calc_method}"
        
        if column_name not in df.columns:
            print(f"⚠️ 列 '{column_name}' が見つかりません。スキップします。")
            continue
            
        print(f"\n--- Channel {ch_num} 褪色カーブ推定 ---")
        
        y_raw = df[column_name].to_numpy()
        # SGフィルタ適用（ウィンドウサイズ13, 多項式次数1）
        y_smooth = savgol_filter(y_raw, window_length=13, polyorder=1, mode='interp')
        
        # 初期値と上限の推定
        a0 = c0 = y_smooth[0] / 2
        p0 = [a0, 0.01, c0, 0.001]
        y_max = np.nanmax(y_smooth)
        upper_limit = y_max * scale_margin
        bounds = ([0, 0, 0, 0], [upper_limit, 1, upper_limit, 1])
        
        try:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                popt, _ = curve_fit(double_exp, t, y_smooth, p0=p0, bounds=bounds, maxfev=10000)
            
            fitted = double_exp(t, *popt)
            scale = double_exp(0, *popt) / fitted  # t=0で正規化
            
            bleach_params[ch_num] = popt
            scale_factors[ch_num] = scale
            
            print(f"✅ Ch{ch_num} フィッティング完了: a={popt[0]:.2f}, b={popt[1]:.4f}, c={popt[2]:.2f}, d={popt[3]:.4f}")
            
        except Exception as e:
            print(f"❌ Ch{ch_num} フィッティング失敗: {e}")
            # フォールバック: スケールファクター=1（補正なし）
            scale_factors[ch_num] = np.ones_like(t)
    
    # プロット作成・保存
    if bleach_params:
        try:
            fig, axes = plt.subplots(1, len(channel_numbers), figsize=(6 * len(channel_numbers), 4), sharey=True)
            if len(channel_numbers) == 1:
                axes = [axes]
            
            for idx, ch in enumerate(channel_numbers):
                col_name = f"ch{ch}{calc_method}"
                y = df[col_name].to_numpy()
                if ch in bleach_params:
                    fit = double_exp(t, *bleach_params[ch])
                    corrected = y * scale_factors[ch]
                else:
                    fit = np.ones_like(t) * y[0]
                    corrected = y
                    
                axes[idx].plot(t, y, 'o', label=f'Ch{ch} Raw', markersize=2)
                axes[idx].plot(t, fit, '-', label=f'Ch{ch} Fit', linewidth=2)
                axes[idx].plot(t, corrected, '--', label=f'Ch{ch} Corrected', linewidth=2)
                axes[idx].set_title(f'Channel {ch} Bleach Curve')
                axes[idx].set_xlabel('Time')
                axes[idx].legend()
                
            axes[0].set_ylabel('Intensity')
            plt.suptitle(f'{file_name} - Bleach Correction')
            plt.tight_layout()
            
            # ファイル名に対応した保存
            plot_filename = f'bleach_curve_{file_name}{calc_method}.png'
            plt.savefig(plot_filename, dpi=150, bbox_inches='tight')
            print(f"📊 プロット保存: {plot_filename}")
            plt.show()
            
        except Exception as plot_error:
            print(f"⚠️ プロット作成エラー: {plot_error}")
    
    return scale_factors

# GPU可用性チェック
try:
    GPU_AVAILABLE = True
    gpu_props = cp.cuda.runtime.getDeviceProperties(0)
    gpu_name = gpu_props['name'].decode('utf-8')
    total_memory_gb = cp.cuda.Device().mem_info[1] / (1024**3)
    print("🚀 GPU処理が利用可能です")
    print(f"🎮 GPU: {gpu_name}")
    print(f"💾 VRAM: {total_memory_gb:.1f}GB")
except ImportError:
    GPU_AVAILABLE = False
    print("⚠️ CuPyが利用できません。CPU処理を使用します")

def process_with_gpu_multichannel(h5_input_path, h5_output_path, scale_factors_all, file_name):
    """チャンネル別褪色補正GPU処理（全ボリューム対応）"""
    if not GPU_AVAILABLE:
        print("❌ GPU処理をスキップします")
        return False
    
    if not scale_factors_all:
        print("❌ スケールファクターが利用できません")
        return False
    
    print(f"\n🎮 {file_name} のGPU褪色補正処理開始...")
    start_time = time.time()
    
    try:
        with h5py.File(h5_input_path, "r") as f_in, h5py.File(h5_output_path, "w") as f_out:
            dset_in = f_in["/default"]
            T_full, C, Z, Y, X = dset_in.shape
            
            print(f"📐 データ形状: (T={T_full}, C={C}, Z={Z}, Y={Y}, X={X})")
            print(f"🎯 処理対象: 全ボリューム（{T_full}フレーム）")
            
            # 出力データセット作成（全ボリューム）
            dset_out = f_out.create_dataset(
                "/default", 
                shape=(T_full, C, Z, Y, X),
                dtype='uint16',
                chunks=(min(chunk_size//4, 20), 1, Z, Y, X),
                compression="gzip",
                compression_opts=1
            )
            
            # メタデータ追加
            dset_out.attrs['processing_mode'] = 'multichannel_bleach_correction_full'
            dset_out.attrs['file_name'] = file_name
            dset_out.attrs['chunk_size'] = chunk_size
            dset_out.attrs['offset_value'] = offset_value
            dset_out.attrs['processed_frames'] = T_full
            dset_out.attrs['frame_range'] = f"0-{T_full-1}"
            
            # チャンネル別スケールファクターの準備
            scale_factors_gpu = {}
            for ch_num in range(C):
                if ch_num in scale_factors_all:
                    # 全フレーム分を取得
                    scale_factors_gpu[ch_num] = cp.asarray(
                        scale_factors_all[ch_num], dtype=cp.float32
                    )
                    print(f"✅ Ch{ch_num}: GPU用スケールファクター準備完了（{T_full}フレーム）")
                else:
                    scale_factors_gpu[ch_num] = cp.ones(T_full, dtype=cp.float32)
                    print(f"⚠️ Ch{ch_num}: スケールファクターなし（補正スキップ）")
            
            # チャンク処理メインループ（全フレーム）
            for t_start in tqdm(range(0, T_full, chunk_size), desc=f"🎮 {file_name} GPU褪色補正"):
                t_end = min(t_start + chunk_size, T_full)
                current_chunk_size = t_end - t_start
                
                try:
                    # メモリ使用量監視
                    memory_before = cp.cuda.Device().mem_info[0] / (1024**3)
                    
                    # チャンネル別処理
                    for c in range(C):
                        # CPUからGPUへデータ転送
                        vol_chunk = cp.asarray(
                            dset_in[t_start:t_end, c, :, :, :], 
                            dtype=cp.float32
                        )
                        
                        # チャンネル専用スケールファクター取得
                        scale_chunk = scale_factors_gpu[c][t_start:t_end]
                        
                        # GPU上でベクトル化処理
                        vol_chunk -= offset_value  # オフセット減算
                        vol_chunk *= scale_chunk.reshape(-1, 1, 1, 1)  # チャンネル別褪色補正
                        vol_chunk = cp.clip(vol_chunk, 0, 65535)  # クリッピング
                        
                        # 型変換と結果保存
                        vol_chunk = vol_chunk.astype(cp.uint16)
                        result = cp.asnumpy(vol_chunk)
                        dset_out[t_start:t_end, c, :, :, :] = result
                        
                        # メモリクリーンアップ（チャンネル毎）
                        del vol_chunk, result
                        cp.cuda.Stream.null.synchronize()
                    
                    # 進捗情報表示
                    if t_start % (chunk_size * 5) == 0:
                        elapsed = time.time() - start_time
                        processed_frames = t_end
                        fps = processed_frames / elapsed if elapsed > 0 else 0
                        remaining_frames = T_full - processed_frames
                        eta_minutes = (remaining_frames / fps / 60) if fps > 0 else 0
                        
                        print(f"  📊 進捗: {processed_frames}/{T_full} ({100*processed_frames/T_full:.1f}%)")
                        print(f"  🚀 速度: {fps:.1f}fps, VRAM: {memory_before:.1f}GB, ETA: {eta_minutes:.1f}分")
                        
                except cp.cuda.memory.OutOfMemoryError:
                    print(f"💥 GPU メモリ不足 (chunk_size={chunk_size})")
                    cp.get_default_memory_pool().free_all_blocks()
                    continue
                    
                except Exception as e:
                    print(f"❌ 処理エラー (t={t_start}-{t_end}): {e}")
                    continue
        
        # 最終メモリクリーンアップ
        try:
            channel_keys = list(scale_factors_gpu.keys())
            for ch_num in channel_keys:
                if ch_num in scale_factors_gpu:
                    del scale_factors_gpu[ch_num]
            cp.get_default_memory_pool().free_all_blocks()
        except Exception as cleanup_error:
            print(f"⚠️ メモリクリーンアップ中にエラー: {cleanup_error}")
            try:
                cp.get_default_memory_pool().free_all_blocks()
            except:
                pass
        
        # 処理結果サマリー
        total_elapsed = time.time() - start_time
        average_fps = T_full / total_elapsed if total_elapsed > 0 else 0
        
        print(f"\n🎉 {file_name} 褪色補正GPU処理完了!")
        print(f"📊 処理フレーム数: {T_full}")
        print(f"🔄 処理チャンネル数: {C}")
        print(f"⏱️  総処理時間: {total_elapsed/60:.1f}分")
        print(f"🚀 平均処理速度: {average_fps:.1f}フレーム/秒")
        print(f"💾 出力ファイル: {h5_output_path}")
        
        return True
        
    except Exception as e:
        print(f"❌ {file_name} 処理中にエラーが発生: {e}")
        return False

# === 複数ファイル処理実行 ===
print("🎯 複数ファイルの褪色補正処理を開始...")
print(f"📁 処理対象ファイル数: {len(file_configs)}")

total_start_time = time.time()
processed_files = 0
failed_files = []

for file_idx, config in enumerate(file_configs, 1):
    print(f"\n{'='*100}")
    print(f"📂 ファイル {file_idx}/{len(file_configs)}: {config['name']}")
    print(f"{'='*100}")
    
    # ファイル存在確認
    if not os.path.exists(config['csv_path']):
        print(f"❌ CSVファイルが見つかりません: {config['csv_path']}")
        failed_files.append(config['name'])
        continue
        
    if not os.path.exists(config['h5_input_path']):
        print(f"❌ 入力H5ファイルが見つかりません: {config['h5_input_path']}")
        failed_files.append(config['name'])
        continue
    
    try:
        # 1. 褪色カーブ推定
        scale_factors_all = estimate_bleach_curves(
            config['csv_path'], 
            config['name']
        )
        
        if scale_factors_all is None:
            print(f"❌ {config['name']}: 褪色カーブ推定に失敗")
            failed_files.append(config['name'])
            continue
        
        print(f"\n✅ {config['name']}: 全チャンネルの褪色カーブ推定完了")
        
        # 2. 出力パス設定
        h5_output_path = config['h5_output_path'].format(calc_method)
        
        # 3. GPU褪色補正処理
        if GPU_AVAILABLE and scale_factors_all:
            success = process_with_gpu_multichannel(
                config['h5_input_path'],
                h5_output_path,
                scale_factors_all,
                config['name']
            )
            
            if success:
                processed_files += 1
                print(f"✅ ファイル {file_idx} 完了: {config['name']}")
            else:
                failed_files.append(config['name'])
        else:
            print("⚠️ GPU処理またはスケールファクターが利用できません")
            failed_files.append(config['name'])
        
        # メモリクリーンアップ
        del scale_factors_all
        if GPU_AVAILABLE:
            cp.get_default_memory_pool().free_all_blocks()
            
    except Exception as e:
        print(f"❌ ファイル {file_idx} 処理エラー: {e}")
        failed_files.append(config['name'])
        continue

# === 全体処理結果サマリー ===
total_elapsed = time.time() - total_start_time

print(f"\n{'='*100}")
print(f"🎉 全ファイル褪色補正処理完了!")
print(f"{'='*100}")
print(f"📊 処理統計:")
print(f"  ✅ 成功: {processed_files}/{len(file_configs)} ファイル")
print(f"  ❌ 失敗: {len(failed_files)} ファイル")
print(f"⏱️  総処理時間: {total_elapsed/60:.1f}分")

if failed_files:
    print(f"\n⚠️ 失敗したファイル:")
    for failed_file in failed_files:
        print(f"  • {failed_file}")

print("\n🧹 最終メモリクリーンアップ...")
if GPU_AVAILABLE:
    try:
        cp.get_default_memory_pool().free_all_blocks()
        print("🧹 GPU メモリクリーンアップ完了")
    except:
        pass

print("🎯 全褪色補正処理完了!")

########################################################################
# MIP画像の生成
########################################################################
import h5py
import numpy as np
import tifffile as tiff
import os
from tqdm import tqdm
import time

# GPU可用性チェック
try:
    import cupy as cp
    GPU_AVAILABLE = True
    print("🚀 GPU処理が利用可能です")
    
    # GPU情報表示
    gpu_props = cp.cuda.runtime.getDeviceProperties(0)
    gpu_name = gpu_props['name'].decode('utf-8')
    total_memory_gb = cp.cuda.Device().mem_info[1] / (1024**3)
    print(f"🎮 GPU: {gpu_name}")
    print(f"💾 VRAM: {total_memory_gb:.1f}GB")
    
except ImportError:
    GPU_AVAILABLE = False
    print("⚠️ CPU処理を使用します")

# === パラメータ（複数ファイル対応） ===
# 処理対象ファイルの定義
# orthogonal_configs = [
#     {
#         "name": "20240508-200229tdTomato-10mW-4",
#         "hdf5_file": r"I:\20240508-200229tdTomato-10mW-4_raw_gzip\20240508-200229tdTomato-10mW-4_raw_gzip_bleachcorrect_top90_mean.h5",
#         "output_path": r"I:\20240508-200229tdTomato-10mW-4_raw_gzip\orthogonal_views"
#     },
#     {
#         "name": "20240516-203245tdTomato-7mW-3",
#         "hdf5_file": r"I:\20240516-203245tdTomato-7mW-3_raw_gzip\20240516-203245tdTomato-7mW-3_raw_gzip_bleachcorrect_top90_mean.h5",
#         "output_path": r"I:\20240516-203245tdTomato-7mW-3_raw_gzip\orthogonal_views"
#     }
# ]


orthogonal_configs = [
    {
        "name": "20240508-174849tdTomato-12mW-3",
        "hdf5_file": r"I:\20240508-174849tdTomato-12mW-3_raw_gzip\20240508-174849tdTomato-12mW-3_corrected_size_gpu_bleachcorrect_top90_mean.h5",
        "output_path": r"I:\20240508-174849tdTomato-12mW-3_raw_gzip\orthogonal_views"
    }
]

chunk_size = 100 if GPU_AVAILABLE else 50  # GPU使用時はより大きなチャンク

def process_cpu_chunk(chunk_data, t_start, t_end, orthogonal_view, channels, z, y, x):
    """CPU処理用のヘルパー関数"""
    for i, t in enumerate(range(t_start, t_end)):
        for c in range(channels):
            data = chunk_data[i, c]  # (Z, Y, X)
            
            # CPU上で最大投影
            xy_proj = np.max(data, axis=0)  # (Y, X)
            yz_proj = np.max(data, axis=2).T  # (Z, Y)
            xz_proj = np.max(data, axis=1)  # (Z, X)
            
            # 直交ビューへの配置
            orthogonal_view[t, c, 0:y, 0:x] = xy_proj
            orthogonal_view[t, c, 0:y, x+3:x+z+3] = yz_proj
            orthogonal_view[t, c, y+3:y+z+3, 0:x] = xz_proj
            orthogonal_view[t, c, y+z+2, x+z+2] = 1000

def process_orthogonal_projections_single(hdf5_file, output_path, file_name):
    """単一ファイルの直交投影処理"""
    print(f"\n🎯 {file_name} の直交投影処理開始...")
    
    # ファイル存在確認
    if not os.path.exists(hdf5_file):
        print(f"❌ H5ファイルが見つかりません: {hdf5_file}")
        return False
    
    # 出力フォルダ作成
    if not os.path.exists(output_path):
        os.makedirs(output_path)
        print(f"📁 出力フォルダ作成: {output_path}")
    
    dirname = os.path.splitext(os.path.basename(hdf5_file))[0]
    start_time = time.time()
    
    try:
        # === HDF5ファイル情報取得 ===
        with h5py.File(hdf5_file, 'r') as file:
            array = file['default']
            total_volumes, channels, z, y, x = array.shape
            print(f"📐 データ形状: (T={total_volumes}, C={channels}, Z={z}, Y={y}, X={x})")
            print(f"🎯 処理対象: 全ボリューム（{total_volumes}フレーム）")
        
        # === 出力配列の準備（全ボリューム） ===
        orthogonal_view = np.zeros((total_volumes, channels, y + z + 3, x + z + 3), dtype='uint16')
        print(f"💾 メモリ確保: {orthogonal_view.nbytes / (1024**3):.2f}GB")
        
        def process_orthogonal_projections():
            """直交投影処理（CPU/GPU自動選択・全ボリューム）"""
            use_gpu = GPU_AVAILABLE  # ローカル変数として処理状態を管理
            
            with h5py.File(hdf5_file, 'r') as file:
                array = file['default']
                
                for t_start in tqdm(range(0, total_volumes, chunk_size), 
                                   desc=f"🎮 {file_name} GPU直交投影" if use_gpu else f"🖥️ {file_name} CPU直交投影"):
                    t_end = min(t_start + chunk_size, total_volumes)
                    
                    # チャンクデータ読み込み
                    chunk_data = array[t_start:t_end]  # (chunk_size, C, Z, Y, X)
                    
                    if use_gpu:
                        # GPU処理を試行
                        try:
                            chunk_gpu = cp.asarray(chunk_data)
                            
                            for i, t in enumerate(range(t_start, t_end)):
                                for c in range(channels):
                                    data_gpu = chunk_gpu[i, c]  # (Z, Y, X)
                                    
                                    # GPU上で最大投影
                                    xy_proj = cp.max(data_gpu, axis=0)  # (Y, X)
                                    yz_proj = cp.max(data_gpu, axis=2).T  # (Z, Y)
                                    xz_proj = cp.max(data_gpu, axis=1)  # (Z, X)
                                    
                                    # CPU側に結果をコピー
                                    orthogonal_view[t, c, 0:y, 0:x] = cp.asnumpy(xy_proj)
                                    orthogonal_view[t, c, 0:y, x+3:x+z+3] = cp.asnumpy(yz_proj)
                                    orthogonal_view[t, c, y+3:y+z+3, 0:x] = cp.asnumpy(xz_proj)
                                    orthogonal_view[t, c, y+z+2, x+z+2] = 1000
                            
                            # GPUメモリクリーンアップ
                            del chunk_gpu
                            cp.get_default_memory_pool().free_all_blocks()
                            
                            # 進捗情報表示（GPU処理）
                            if t_start % (chunk_size * 5) == 0:
                                processed = t_end
                                progress_pct = 100 * processed / total_volumes
                                elapsed = time.time() - start_time
                                fps = processed / elapsed if elapsed > 0 else 0
                                eta_minutes = ((total_volumes - processed) / fps / 60) if fps > 0 else 0
                                print(f"  📊 GPU進捗: {processed}/{total_volumes} ({progress_pct:.1f}%), 速度: {fps:.1f}fps, ETA: {eta_minutes:.1f}分")
                            
                        except Exception as e:
                            print(f"⚠️ GPU処理エラー、CPUにフォールバック: {e}")
                            use_gpu = False  # 今後のチャンクはCPU処理
                            # CPU処理で再試行
                            process_cpu_chunk(chunk_data, t_start, t_end, orthogonal_view, channels, z, y, x)
                            
                    else:
                        # CPU処理
                        process_cpu_chunk(chunk_data, t_start, t_end, orthogonal_view, channels, z, y, x)
                        
                        # 進捗情報表示（CPU処理）
                        if t_start % (chunk_size * 2) == 0:
                            processed = t_end
                            progress_pct = 100 * processed / total_volumes
                            elapsed = time.time() - start_time
                            fps = processed / elapsed if elapsed > 0 else 0
                            eta_minutes = ((total_volumes - processed) / fps / 60) if fps > 0 else 0
                            print(f"  📊 CPU進捗: {processed}/{total_volumes} ({progress_pct:.1f}%), 速度: {fps:.1f}fps, ETA: {eta_minutes:.1f}分")
        
        # === 処理実行 ===
        print(f"🎯 全ボリューム（{total_volumes}フレーム）の直交投影処理を開始...")
        process_orthogonal_projections()
        print("🎉 直交投影処理完了!")
        
        # === TIFF保存（全ボリューム） ===
        file_name_tiff = f"orthogonal_view_{dirname}_full.tif"
        output_path_full = os.path.join(output_path, file_name_tiff)
        
        print("💾 TIFF保存中...")
        print(f"📁 保存先: {output_path_full}")
        print(f"📊 データサイズ: {orthogonal_view.nbytes / (1024**3):.2f}GB")
        
        try:
            tiff.imwrite(
                output_path_full, 
                orthogonal_view, 
                imagej=True, 
                metadata={'axes': 'TCYX'}, 
                compression='zlib'
            )
            
            # 処理結果サマリー
            total_elapsed = time.time() - start_time
            average_fps = total_volumes / total_elapsed if total_elapsed > 0 else 0
            
            print(f"✅ {file_name} 処理完了: {output_path_full}")
            print(f"📈 出力形状: {orthogonal_view.shape}")
            print(f"⏱️  総処理時間: {total_elapsed/60:.1f}分")
            print(f"🚀 平均処理速度: {average_fps:.1f}フレーム/秒")
            print(f"📊 処理統計: {total_volumes}/{total_volumes}ボリューム (100%)")
            
            return True
            
        except Exception as save_error:
            print(f"❌ TIFF保存エラー: {save_error}")
            print("💡 メモリ不足の可能性があります。チャンク保存を試します...")
            
            # === フォールバック: チャンク保存 ===
            save_chunk_size = 500  # 500フレームずつ保存
            
            for save_start in tqdm(range(0, total_volumes, save_chunk_size), desc="📦 チャンク保存"):
                save_end = min(save_start + save_chunk_size, total_volumes)
                chunk_name = f"orthogonal_view_{dirname}_part{save_start:05d}-{save_end:05d}.tif"
                chunk_path = os.path.join(output_path, chunk_name)
                
                chunk_data = orthogonal_view[save_start:save_end]
                tiff.imwrite(
                    chunk_path,
                    chunk_data,
                    imagej=True,
                    metadata={'axes': 'TCYX'},
                    compression='zlib'
                )
                print(f"✅ 保存完了: {chunk_name}")
            
            print(f"📦 チャンク保存完了: {output_path}")
            return True
        
    except Exception as e:
        print(f"❌ {file_name} 処理中にエラーが発生: {e}")
        return False
    
    finally:
        # === メモリクリーンアップ ===
        try:
            if 'orthogonal_view' in locals():
                del orthogonal_view
            if GPU_AVAILABLE:
                cp.get_default_memory_pool().free_all_blocks()
            print("🧹 メモリクリーンアップ完了")
        except:
            pass

# === 複数ファイル処理実行 ===
print("🎯 複数ファイルの直交投影処理を開始...")
print(f"📁 処理対象ファイル数: {len(orthogonal_configs)}")

total_start_time = time.time()
processed_files = 0
failed_files = []

for file_idx, config in enumerate(orthogonal_configs, 1):
    print(f"\n{'='*100}")
    print(f"📂 ファイル {file_idx}/{len(orthogonal_configs)}: {config['name']}")
    print(f"{'='*100}")
    
    # ファイル存在確認
    if not os.path.exists(config['hdf5_file']):
        print(f"❌ H5ファイルが見つかりません: {config['hdf5_file']}")
        failed_files.append(config['name'])
        continue
    
    try:
        # 単一ファイル処理
        success = process_orthogonal_projections_single(
            config['hdf5_file'],
            config['output_path'],
            config['name']
        )
        
        if success:
            processed_files += 1
            print(f"✅ ファイル {file_idx} 完了: {config['name']}")
        else:
            failed_files.append(config['name'])
        
        # メモリクリーンアップ
        if GPU_AVAILABLE:
            cp.get_default_memory_pool().free_all_blocks()
            
    except Exception as e:
        print(f"❌ ファイル {file_idx} 処理エラー: {e}")
        failed_files.append(config['name'])
        continue

# === 全体処理結果サマリー ===
total_elapsed = time.time() - total_start_time

print(f"\n{'='*100}")
print(f"🎉 全ファイル直交投影処理完了!")
print(f"{'='*100}")
print(f"📊 処理統計:")
print(f"  ✅ 成功: {processed_files}/{len(orthogonal_configs)} ファイル")
print(f"  ❌ 失敗: {len(failed_files)} ファイル")
print(f"⏱️  総処理時間: {total_elapsed/60:.1f}分")

if failed_files:
    print(f"\n⚠️ 失敗したファイル:")
    for failed_file in failed_files:
        print(f"  • {failed_file}")

print("\n🧹 最終メモリクリーンアップ...")
if GPU_AVAILABLE:
    try:
        cp.get_default_memory_pool().free_all_blocks()
        print("🧹 GPU メモリクリーンアップ完了")
    except:
        pass

print("🎯 全直交投影処理完了!")


For real-time tracking. Save each volume data as TIFF after bleach correction.

In [ ]:
import h5py
import numpy as np
import tifffile as tiff
import os
from tqdm import tqdm
import time

# === パラメータ ===
hdf5_file = r"I:\20240508-174849tdTomato-12mW-3_raw_gzip\20240508-174849tdTomato-12mW-3_corrected_size_gpu_bleachcorrect_top90_mean.h5"
output_path = r"I:\20240508-174849tdTomato-12mW-3_raw_gzip\bleachcorrect\images_resize"
channel_to_save = 0  # チャンネル0のみ保存
max_volumes = 4900  # 保存する最大ボリューム数（0-4899の4900個）
chunk_size = 50  # I/O効率化のためのチャンクサイズ

# 出力フォルダ作成
if not os.path.exists(output_path):
    os.makedirs(output_path)
    print(f"📁 出力フォルダ作成: {output_path}")

def save_individual_volumes():
    """チャンネル0の各ボリュームを個別TIFFファイルとして保存（CPU版）"""
    start_time = time.time()
    
    # ファイル存在確認
    if not os.path.exists(hdf5_file):
        print(f"❌ H5ファイルが見つかりません: {hdf5_file}")
        return False
    
    with h5py.File(hdf5_file, 'r') as file:
        dset = file['default']
        total_volumes, channels, z, y, x = dset.shape
        
        # 保存対象ボリューム数を制限
        volumes_to_save = min(max_volumes, total_volumes)
        
        print(f"📐 データ形状: (T={total_volumes}, C={channels}, Z={z}, Y={y}, X={x})")
        print(f"🎯 処理対象: チャンネル{channel_to_save} の0-{volumes_to_save-1}ボリューム（{volumes_to_save}個）")
        print(f"💾 各ボリュームサイズ: ({z}, {y}, {x})")
        
        # チャンネル存在確認
        if channel_to_save >= channels:
            print(f"❌ チャンネル{channel_to_save}は存在しません（利用可能: 0-{channels-1}）")
            return False
        
        # チャンク処理でI/O効率化
        saved_count = 0
        failed_count = 0
        
        for t_start in tqdm(range(0, volumes_to_save, chunk_size), 
                           desc=f"💾 Ch{channel_to_save}ボリューム保存（0-{volumes_to_save-1}）"):
            t_end = min(t_start + chunk_size, volumes_to_save)
            
            try:
                # チャンクデータ読み込み (chunk_size, Z, Y, X)
                chunk_data = dset[t_start:t_end, channel_to_save, :, :, :]
                
                # 各ボリュームを個別に保存
                for i, t in enumerate(range(t_start, t_end)):
                    volume_data = chunk_data[i]  # (Z, Y, X)
                    
                    # ファイル名生成（ゼロパディングなし）
                    tiff_filename = f"corrected_volume_t{t}.tiff"
                    tiff_path = os.path.join(output_path, tiff_filename)
                    
                    try:
                        # TIFF保存（CPU処理）
                        tiff.imwrite(
                            tiff_path,
                            volume_data,
                            imagej=True,
                            metadata={'axes': 'ZYX'},
                            compression='zlib'
                        )
                        
                        saved_count += 1
                        
                        # 進捗情報（詳細）
                        if t % 100 == 0:  # 100ボリューム毎に詳細情報表示
                            elapsed = time.time() - start_time
                            fps = saved_count / elapsed if elapsed > 0 else 0
                            remaining = volumes_to_save - saved_count
                            eta_minutes = (remaining / fps / 60) if fps > 0 else 0
                            
                            print(f"  📊 進捗: {saved_count}/{volumes_to_save} ({100*saved_count/volumes_to_save:.1f}%)")
                            print(f"  💾 速度: {fps:.1f}ボリューム/秒, ETA: {eta_minutes:.1f}分")
                            print(f"  📄 最新保存: {tiff_filename}")
                        
                    except Exception as save_error:
                        print(f"❌ ボリューム t{t} 保存エラー: {save_error}")
                        failed_count += 1
                        continue
                
                # メモリクリーンアップ
                del chunk_data
                
            except Exception as chunk_error:
                print(f"❌ チャンク t{t_start}-{t_end} 読み込みエラー: {chunk_error}")
                failed_count += chunk_size
                continue
    
    # 処理結果サマリー
    total_elapsed = time.time() - start_time
    average_fps = saved_count / total_elapsed if total_elapsed > 0 else 0
    
    print(f"\n🎉 個別ボリューム保存完了!")
    print(f"📊 保存統計:")
    print(f"  ✅ 成功: {saved_count}/{volumes_to_save} ボリューム")
    print(f"  ❌ 失敗: {failed_count} ボリューム")
    print(f"⏱️  総処理時間: {total_elapsed/60:.1f}分")
    print(f"💾 平均保存速度: {average_fps:.1f}ボリューム/秒")
    print(f"📁 保存先: {output_path}")
    print(f"📄 ファイル形式: corrected_volume_t0.tiff ～ corrected_volume_t{volumes_to_save-1}.tiff")
    
    # 保存されたファイルのサンプル表示
    print(f"\n📋 保存ファイル例:")
    sample_files = []
    for t in [0, 1, 2, 10, 100, 1000, volumes_to_save//2, volumes_to_save-1]:
        if t < volumes_to_save:
            filename = f"corrected_volume_t{t}.tiff"
            filepath = os.path.join(output_path, filename)
            if os.path.exists(filepath):
                file_size_mb = os.path.getsize(filepath) / (1024**2)
                sample_files.append(f"  📄 {filename} ({file_size_mb:.1f}MB)")
    
    for sample in sample_files[:10]:  # 最大10個表示
        print(sample)
    
    if len(sample_files) > 10:
        print(f"  ... 他 {len(sample_files)-10} ファイル")
    
    return saved_count == volumes_to_save

# === 実行 ===
print(f"🎯 チャンネル0の個別ボリューム保存を開始（0-{max_volumes-1}ボリューム）...")
print("💡 CPU版（ファイル保存に最適化）")
success = save_individual_volumes()

if success:
    print("✅ 指定範囲のボリューム保存が正常に完了しました！")
else:
    print("⚠️ 一部のボリューム保存でエラーが発生しました。ログを確認してください。")

For spinning disk confocal microscope data. Make voxel size isotropic.

In [ ]:
import h5py
import numpy as np
from scipy.ndimage import zoom
from tqdm import tqdm
import time
import os

# GPU可用性チェック（大容量データ処理用）
try:
    import cupy as cp
    from cupyx.scipy.ndimage import zoom as cp_zoom
    GPU_AVAILABLE = True
    print("🚀 GPU処理が利用可能です")
    
    # GPU情報表示
    gpu_props = cp.cuda.runtime.getDeviceProperties(0)
    gpu_name = gpu_props['name'].decode('utf-8')
    total_memory_gb = cp.cuda.Device().mem_info[1] / (1024**3)
    print(f"🎮 GPU: {gpu_name}, VRAM: {total_memory_gb:.1f}GB")
    
except ImportError:
    GPU_AVAILABLE = False
    print("⚠️ CPU処理を使用します（大容量データの場合時間がかかります）")

# === パラメータ ===
input_h5_path = r"I:\20240911_cam2_007_raw\20240911_cam2_007_raw.h5"
output_h5_path = r"I:\20240911_cam2_007_raw\20240911_cam2_007_raw_affine.h5"

# 現在のvoxelサイズ
original_voxel_size = {
    'x': 0.43,  # μm
    'y': 0.43,  # μm  
    'z': 2.0    # μm
}

# 目標voxelサイズ（等方的）
target_voxel_size = 0.43  # μm (xyz全て同じ)

# チャンクサイズ（メモリ使用量に応じて調整）
chunk_size = 20 if GPU_AVAILABLE else 10

def calculate_zoom_factors():
    """ズーム倍率を計算"""
    zoom_factors = {
        'x': original_voxel_size['x'] / target_voxel_size,
        'y': original_voxel_size['y'] / target_voxel_size,
        'z': original_voxel_size['z'] / target_voxel_size
    }
    
    print(f"📐 元のvoxelサイズ: x={original_voxel_size['x']}μm, y={original_voxel_size['y']}μm, z={original_voxel_size['z']}μm")
    print(f"🎯 目標voxelサイズ: {target_voxel_size}μm (等方的)")
    print(f"🔍 ズーム倍率: x={zoom_factors['x']:.3f}, y={zoom_factors['y']:.3f}, z={zoom_factors['z']:.3f}")
    
    return zoom_factors

def calculate_output_shape(input_shape, zoom_factors):
    """出力データサイズを計算"""
    t, z, y, x = input_shape
    
    new_z = int(z * zoom_factors['z'])
    new_y = int(y * zoom_factors['y'])
    new_x = int(x * zoom_factors['x'])
    
    print(f"📏 元のデータ形状: (T={t}, Z={z}, Y={y}, X={x})")
    print(f"📏 出力データ形状: (T={t}, Z={new_z}, Y={new_y}, X={new_x})")
    print(f"📊 データサイズ変化: {t*z*y*x:,} → {t*new_z*new_y*new_x:,} voxels ({new_z*new_y*new_x/(z*y*x):.2f}倍)")
    
    return (t, new_z, new_y, new_x)

def process_volume_gpu(volume_data, zoom_factors):
    """GPU版ボリューム補間処理"""
    try:
        # CPUからGPUへデータ転送
        volume_gpu = cp.asarray(volume_data, dtype=cp.float32)
        
        # 3D線形補間（Z, Y, X軸）
        zoom_scales = (zoom_factors['z'], zoom_factors['y'], zoom_factors['x'])
        interpolated_gpu = cp_zoom(volume_gpu, zoom_scales, order=1, mode='constant', cval=0)
        
        # データ型を元に戻してCPUに転送
        result = cp.asnumpy(interpolated_gpu.astype(cp.uint16))
        
        # GPUメモリクリーンアップ
        del volume_gpu, interpolated_gpu
        cp.get_default_memory_pool().free_all_blocks()
        
        return result
        
    except Exception as e:
        print(f"⚠️ GPU処理エラー、CPUにフォールバック: {e}")
        return process_volume_cpu(volume_data, zoom_factors)

def process_volume_cpu(volume_data, zoom_factors):
    """CPU版ボリューム補間処理"""
    # 3D線形補間（Z, Y, X軸）
    zoom_scales = (zoom_factors['z'], zoom_factors['y'], zoom_factors['x'])
    interpolated = zoom(volume_data.astype(np.float32), zoom_scales, order=1, mode='constant', cval=0)
    
    return interpolated.astype(np.uint16)

def create_isotropic_voxel_data():
    """等方的voxelサイズのHDF5ファイルを作成"""
    start_time = time.time()
    
    # ファイル存在確認
    if not os.path.exists(input_h5_path):
        print(f"❌ 入力ファイルが見つかりません: {input_h5_path}")
        return False
    
    # ズーム倍率計算
    zoom_factors = calculate_zoom_factors()
    
    with h5py.File(input_h5_path, 'r') as f_in:
        dset_in = f_in['/default']
        input_shape = dset_in.shape  # (T, Z, Y, X)
        
        # 出力サイズ計算
        output_shape = calculate_output_shape(input_shape, zoom_factors)
        t_len, new_z, new_y, new_x = output_shape
        
        # メモリ使用量推定
        input_size_gb = (np.prod(input_shape) * 2) / (1024**3)  # uint16 = 2bytes
        output_size_gb = (np.prod(output_shape) * 2) / (1024**3)
        
        print(f"💾 推定メモリ使用量:")
        print(f"  入力データ: {input_size_gb:.2f}GB")
        print(f"  出力データ: {output_size_gb:.2f}GB")
        print(f"  処理チャンクサイズ: {chunk_size}フレーム")
        
        # 出力ファイル作成
        with h5py.File(output_h5_path, 'w') as f_out:
            # 出力データセット作成
            dset_out = f_out.create_dataset(
                '/default',
                shape=output_shape,
                dtype='uint16',
                chunks=(1, new_z, new_y, new_x),
                compression='gzip',
                compression_opts=1
            )
            
            # メタデータ追加
            dset_out.attrs['original_voxel_size_x_um'] = original_voxel_size['x']
            dset_out.attrs['original_voxel_size_y_um'] = original_voxel_size['y']
            dset_out.attrs['original_voxel_size_z_um'] = original_voxel_size['z']
            dset_out.attrs['target_voxel_size_um'] = target_voxel_size
            dset_out.attrs['zoom_factor_x'] = zoom_factors['x']
            dset_out.attrs['zoom_factor_y'] = zoom_factors['y']
            dset_out.attrs['zoom_factor_z'] = zoom_factors['z']
            dset_out.attrs['interpolation_method'] = 'linear'
            dset_out.attrs['processing_mode'] = 'isotropic_voxel_resampling'
            dset_out.attrs['original_shape'] = input_shape
            dset_out.attrs['resampled_shape'] = output_shape
            
            # チャンク処理でメモリ効率化
            processed_frames = 0
            
            for t_start in tqdm(range(0, t_len, chunk_size), 
                               desc=f"🔄 等方的voxel変換 ({'GPU' if GPU_AVAILABLE else 'CPU'})"):
                t_end = min(t_start + chunk_size, t_len)
                
                try:
                    # チャンクデータ読み込み
                    chunk_data = dset_in[t_start:t_end]  # (chunk_size, Z, Y, X)
                    
                    # 各フレームを個別に処理
                    for i, t in enumerate(range(t_start, t_end)):
                        volume = chunk_data[i]  # (Z, Y, X)
                        
                        # GPU/CPU処理の選択
                        if GPU_AVAILABLE:
                            resampled_volume = process_volume_gpu(volume, zoom_factors)
                        else:
                            resampled_volume = process_volume_cpu(volume, zoom_factors)
                        
                        # 結果保存
                        dset_out[t] = resampled_volume
                        processed_frames += 1
                        
                        # 進捗情報（詳細）
                        if processed_frames % 50 == 0:
                            elapsed = time.time() - start_time
                            fps = processed_frames / elapsed if elapsed > 0 else 0
                            remaining = t_len - processed_frames
                            eta_minutes = (remaining / fps / 60) if fps > 0 else 0
                            
                            print(f"  📊 進捗: {processed_frames}/{t_len} ({100*processed_frames/t_len:.1f}%)")
                            print(f"  🚀 速度: {fps:.1f}フレーム/秒, ETA: {eta_minutes:.1f}分")
                    
                    # メモリクリーンアップ
                    del chunk_data
                    if GPU_AVAILABLE:
                        cp.get_default_memory_pool().free_all_blocks()
                        
                except Exception as chunk_error:
                    print(f"❌ チャンク t{t_start}-{t_end} 処理エラー: {chunk_error}")
                    continue
    
    # 処理結果サマリー
    total_elapsed = time.time() - start_time
    average_fps = processed_frames / total_elapsed if total_elapsed > 0 else 0
    
    print(f"\n🎉 等方的voxelサイズ変換完了!")
    print(f"📊 処理統計:")
    print(f"  ✅ 処理フレーム数: {processed_frames}/{t_len}")
    print(f"  ⏱️  総処理時間: {total_elapsed/60:.1f}分")
    print(f"  🚀 平均処理速度: {average_fps:.1f}フレーム/秒")
    print(f"  📏 voxelサイズ変更: {original_voxel_size} → {target_voxel_size}μm (等方的)")
    print(f"  📁 出力ファイル: {output_h5_path}")
    
    # 出力ファイルサイズ確認
    if os.path.exists(output_h5_path):
        file_size_gb = os.path.getsize(output_h5_path) / (1024**3)
        print(f"  💾 出力ファイルサイズ: {file_size_gb:.2f}GB")
    
    return processed_frames == t_len

# === 実行 ===
print("🎯 スピニング共焦点データの等方的voxelサイズ変換を開始...")
print(f"📂 入力: {os.path.basename(input_h5_path)}")
print(f"📂 出力: {os.path.basename(output_h5_path)}")

success = create_isotropic_voxel_data()

if success:
    print("\n✅ 等方的voxelサイズ変換が正常に完了しました！")
    print("💡 次のステップ:")
    print("   1. 変換後データの品質確認")
    print("   2. 必要に応じて上位90%平均値の再計算")
    print("   3. 褪色補正処理の実行")
else:
    print("\n⚠️ 処理中にエラーが発生しました。ログを確認してください。")

# === 最終メモリクリーンアップ ===
if GPU_AVAILABLE:
    try:
        cp.get_default_memory_pool().free_all_blocks()
        print("🧹 GPU メモリクリーンアップ完了")
    except:
        pass

For spinning disk confocal microscope data. Compute the top 90% of the data.

In [ ]:
import h5py
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import time

# GPU可用性チェック
try:
    import cupy as cp
    GPU_AVAILABLE = True
    print("🚀 GPU処理が利用可能です")
    
    # GPU情報表示
    gpu_props = cp.cuda.runtime.getDeviceProperties(0)
    gpu_name = gpu_props['name'].decode('utf-8')
    total_memory_gb = cp.cuda.Device().mem_info[1] / (1024**3)
    print(f"🎮 GPU: {gpu_name}, VRAM: {total_memory_gb:.1f}GB")
    
except ImportError:
    GPU_AVAILABLE = False
    print("⚠️ CPU処理を使用します")

# === パラメータ ===
h5_path = r"I:\20240911_cam2_007_raw\20240911_cam2_007_raw_affine.h5"
dataset_name = "/default"
output_csv = os.path.splitext(h5_path)[0] + "_top90_mean.csv"

# 高速化パラメータ
offset_value = 1600
exclude_value = -1600
chunk_size = 50 if GPU_AVAILABLE else 25  # GPU使用時はより大きなチャンク

def process_volume_gpu(vol_chunk, offset_value, exclude_value):
    """GPU用のボリューム統計処理 (T, Z, Y, X)形状対応"""
    try:
        # GPUに転送
        vol_gpu = cp.asarray(vol_chunk, dtype=cp.float32)
        vol_gpu -= offset_value
        
        # 各統計の計算
        stats = {}
        
        # -100除外統計
        valid_mask = vol_gpu != exclude_value
        valid_vals = vol_gpu[valid_mask]
        
        # 上位90%の平均値
        percentiles = [90]
        for p in percentiles:
            if len(valid_vals) > 0:
                thresh = cp.percentile(valid_vals, p)
                top_vals = valid_vals[valid_vals >= thresh]
                stats[f'top{p}_mean'] = float(cp.mean(top_vals)) if len(top_vals) > 0 else 0.0
            else:
                stats[f'top{p}_mean'] = 0.0
        
        # メモリクリーンアップ
        del vol_gpu, valid_vals, top_vals, thresh
        
        return stats
        
    except Exception as e:
        print(f"⚠️ GPU処理エラー: {e}")
        # CPUフォールバック
        return process_volume_cpu(vol_chunk, offset_value, exclude_value)

def process_volume_cpu(vol_chunk, offset_value, exclude_value):
    """CPU用のボリューム統計処理 (T, Z, Y, X)形状対応"""
    vol = vol_chunk.astype(np.float32)
    vol -= offset_value
    
    stats = {}
    
    # -100除外統計（ベクトル化）
    valid_mask = vol != exclude_value
    if np.any(valid_mask):
        valid_vals = vol[valid_mask]
        
        # 上位90%の平均値
        percentiles = [90]
        for p in percentiles:
            thresh = np.percentile(valid_vals, p)
            top_vals = valid_vals[valid_vals >= thresh]
            stats[f'top{p}_mean'] = float(np.mean(top_vals)) if len(top_vals) > 0 else 0.0
    else:
        stats['top90_mean'] = 0.0
    
    return stats

def process_statistics_optimized():
    """最適化された統計処理（チャンク+GPU/CPU）- (T, Z, Y, X)形状対応"""
    start_time = time.time()
    
    with h5py.File(h5_path, "r") as f:
        dset = f[dataset_name]
        t_len, z_len, y_len, x_len = dset.shape  # (T, Z, Y, X)
        print(f"📐 Dataset shape: {dset.shape} (T, Z, Y, X)")
        print(f"🚀 処理モード: {'GPU' if GPU_AVAILABLE else 'CPU'}")
        print(f"📦 チャンクサイズ: {chunk_size}")
        
        # 結果辞書の初期化（チャンネルなし）
        result = {"frame": list(range(t_len))}
        result["top90_mean"] = []
        
        # チャンク処理でメモリ効率化
        for t_start in tqdm(range(0, t_len, chunk_size), desc="🎯 統計計算処理"):
            t_end = min(t_start + chunk_size, t_len)
            
            # 複数フレームを一括読み込み (chunk_size, Z, Y, X)
            chunk_data = dset[t_start:t_end]
            
            for i, t in enumerate(range(t_start, t_end)):
                vol_chunk = chunk_data[i]  # shape: (Z, Y, X)
                
                # GPU/CPU処理の選択
                if GPU_AVAILABLE:
                    stats = process_volume_gpu(vol_chunk, offset_value, exclude_value)
                else:
                    stats = process_volume_cpu(vol_chunk, offset_value, exclude_value)
                
                # 結果の蓄積
                result["top90_mean"].append(stats['top90_mean'])
            
            # 進捗情報表示
            if t_start % (chunk_size * 10) == 0:
                elapsed = time.time() - start_time
                processed = t_end
                fps = processed / elapsed if elapsed > 0 else 0
                remaining = t_len - processed
                eta_minutes = (remaining / fps / 60) if fps > 0 else 0
                
                print(f"  📊 進捗: {processed}/{t_len} ({100*processed/t_len:.1f}%)")
                print(f"  🚀 速度: {fps:.1f}フレーム/秒, ETA: {eta_minutes:.1f}分")
            
            # GPUメモリクリーンアップ
            if GPU_AVAILABLE:
                try:
                    cp.get_default_memory_pool().free_all_blocks()
                except:
                    pass
    
    # 処理結果サマリー
    total_elapsed = time.time() - start_time
    average_fps = t_len / total_elapsed if total_elapsed > 0 else 0
    
    print(f"\n🎉 統計計算完了!")
    print(f"📊 処理フレーム数: {t_len}")
    print(f"🔄 チャンネル数: 1 (単一チャンネル)")
    print(f"⏱️ 総処理時間: {total_elapsed:.1f}秒 ({total_elapsed/60:.1f}分)")
    print(f"🚀 平均処理速度: {average_fps:.1f}フレーム/秒")
    
    return result

# === 最適化処理実行 ===
print("🎯 最適化された統計計算を開始...")
print(f"📁 処理対象: {os.path.basename(h5_path)}")
result = process_statistics_optimized()

# === 保存処理 ===
print("💾 CSV保存中...")
df = pd.DataFrame(result)

try:
    df.to_csv(output_csv, index=False)
    print(f"✅ 最適化統計処理完了: {output_csv}")
    print(f"📈 出力データ形状: {df.shape}")
    
    # データサンプル表示
    print("\n📊 データサンプル:")
    print(df.head())
    
except PermissionError:
    print("❌ 書き込みに失敗しました。ファイルが開かれているか、書き込み権限がありません:")
    print("🔒 ファイルを閉じているか、別の保存先を指定してください。")
    print("📄 試行した出力先:", output_csv)
    
    # 代替保存先の提案
    alternative_path = output_csv.replace(".csv", f"_backup_{int(time.time())}.csv")
    try:
        df.to_csv(alternative_path, index=False)
        print(f"✅ 代替パスに保存完了: {alternative_path}")
    except Exception as e:
        print(f"❌ 代替保存も失敗: {e}")

# === メモリクリーンアップ ===
try:
    del result, df
    if GPU_AVAILABLE:
        cp.get_default_memory_pool().free_all_blocks()
    print("🧹 メモリクリーンアップ完了")
except:
    pass

For spinning disk confocal microscope data. Bleach correction.

In [ ]:
import h5py
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from tqdm import tqdm
import warnings
import time
import cupy as cp
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt

# === パラメータ ===
calc_method = "_top90_mean"  # 使用する列のメソッド
csv_path = r"I:\20240911_cam2_007_raw\20240911_cam2_007_raw_top90_mean.csv"
h5_input_path = r"I:\20240911_cam2_007_raw\20240911_cam2_007_raw_affine.h5"
h5_output_path = fr"I:\20240911_cam2_007_raw\20240911_cam2_007_raw_bleachcorrect{calc_method}_affine.h5"
offset_value = 1600
scale_margin = 1.2
chunk_size = 60  # メモリーサイズに応じて設定

# === 褪色関数定義 ===
def double_exp(t, a, b, c, d):
    return a * np.exp(-b * t) + c * np.exp(-d * t)

# === 単一チャンネル褪色カーブ推定 ===
def estimate_bleach_curves():
    """単一チャンネルの褪色カーブを推定"""
    df = pd.read_csv(csv_path)
    t = np.arange(len(df))
    
    bleach_params = {}
    scale_factors = {}
    
    # 単一チャンネル列の検出
    column_name = "top90_mean"  # (T, Z, Y, X)形状では単一チャンネル
    
    if column_name not in df.columns:
        print(f"❌ 列 '{column_name}' が見つかりません。")
        return None
        
    print(f"\n--- 単一チャンネル褪色カーブ推定 ---")
    
    y_raw = df[column_name].to_numpy()
    # SGフィルタ適用（ウィンドウサイズ13, 多項式次数1）
    y_smooth = savgol_filter(y_raw, window_length=13, polyorder=1, mode='interp')
    
    # 初期値と上限の推定
    a0 = c0 = y_smooth[0] / 2
    p0 = [a0, 0.01, c0, 0.001]
    y_max = np.nanmax(y_smooth)
    upper_limit = y_max * scale_margin
    bounds = ([0, 0, 0, 0], [upper_limit, 1, upper_limit, 1])
    
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            popt, _ = curve_fit(double_exp, t, y_smooth, p0=p0, bounds=bounds, maxfev=10000)
        
        fitted = double_exp(t, *popt)
        scale = double_exp(0, *popt) / fitted  # t=0で正規化
        
        bleach_params[0] = popt  # チャンネル0として保存
        scale_factors[0] = scale
        
        # プロット作成・保存（単一チャンネル）
        fig, ax = plt.subplots(1, 1, figsize=(8, 6))
        
        fit = double_exp(t, *popt)
        corrected = y_raw * scale
        
        ax.plot(t, y_raw, 'o', label='Raw Data', markersize=2, alpha=0.7)
        ax.plot(t, fit, '-', label='Fitted Curve', linewidth=2, color='red')
        ax.plot(t, corrected, '--', label='Corrected Data', linewidth=2, color='green')
        ax.set_title('Single Channel Bleach Curve')
        ax.set_xlabel('Time (frames)')
        ax.set_ylabel('Intensity')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(f'bleach_curve_single_channel{calc_method}.png', dpi=150, bbox_inches='tight')
        plt.show()
        
        print(f"✅ 褪色補正フィッティング完了: a={popt[0]:.2f}, b={popt[1]:.4f}, c={popt[2]:.2f}, d={popt[3]:.4f}")
        
    except Exception as e:
        print(f"❌ フィッティング失敗: {e}")
        # フォールバック: スケールファクター=1（補正なし）
        scale_factors[0] = np.ones_like(t)
    
    return scale_factors

# 褪色カーブ推定実行
scale_factors_all = estimate_bleach_curves()
if scale_factors_all:
    print("\n✅ 褪色カーブ推定完了")
else:
    print("\n❌ 褪色カーブ推定に失敗しました")

# GPU版（単一チャンネル対応）
try:
    GPU_AVAILABLE = True
    print("🚀 GPU処理が利用可能です")
    
    # GPU情報表示
    gpu_props = cp.cuda.runtime.getDeviceProperties(0)
    gpu_name = gpu_props['name'].decode('utf-8')
    total_memory_gb = cp.cuda.Device().mem_info[1] / (1024**3)
    print(f"🎮 GPU: {gpu_name}")
    print(f"💾 VRAM: {total_memory_gb:.1f}GB")
    
except ImportError:
    GPU_AVAILABLE = False
    print("⚠️ CuPyが利用できません。CPU処理を使用します")

def process_with_gpu_single_channel():
    """単一チャンネル褪色補正GPU処理（(T, Z, Y, X)形状対応）"""
    if not GPU_AVAILABLE:
        print("❌ GPU処理をスキップします")
        return
    
    start_time = time.time()
    
    with h5py.File(h5_input_path, "r") as f_in, h5py.File(h5_output_path, "w") as f_out:
        dset_in = f_in["/default"]
        T_full, Z, Y, X = dset_in.shape  # (T, Z, Y, X)
        
        print(f"📐 データ形状: (T={T_full}, Z={Z}, Y={Y}, X={X})")
        print(f"🎯 処理対象: 全ボリューム（{T_full}フレーム）")
        
        # 出力データセット作成（全ボリューム）
        dset_out = f_out.create_dataset(
            "/default", 
            shape=(T_full, Z, Y, X),  # 単一チャンネル
            dtype='uint16',
            chunks=(min(chunk_size//4, 20), Z, Y, X),
            compression="gzip",
            compression_opts=1
        )
        
        # メタデータ追加
        dset_out.attrs['processing_mode'] = 'single_channel_bleach_correction_full'
        dset_out.attrs['chunk_size'] = chunk_size
        dset_out.attrs['offset_value'] = offset_value
        dset_out.attrs['processed_frames'] = T_full
        dset_out.attrs['frame_range'] = f"0-{T_full-1}"
        
        # スケールファクターの準備
        if 0 in scale_factors_all:
            scale_factors_gpu = cp.asarray(scale_factors_all[0], dtype=cp.float32)
            print(f"✅ GPU用スケールファクター準備完了（{T_full}フレーム）")
        else:
            scale_factors_gpu = cp.ones(T_full, dtype=cp.float32)
            print(f"⚠️ スケールファクターなし（補正スキップ）")
        
        # チャンク処理メインループ（全フレーム）
        for t_start in tqdm(range(0, T_full, chunk_size), desc="🎮 GPU単一チャンネル褪色補正（全ボリューム）"):
            t_end = min(t_start + chunk_size, T_full)
            current_chunk_size = t_end - t_start
            
            try:
                # メモリ使用量監視
                memory_before = cp.cuda.Device().mem_info[0] / (1024**3)
                
                # CPUからGPUへデータ転送
                vol_chunk = cp.asarray(
                    dset_in[t_start:t_end, :, :, :],  # (chunk_size, Z, Y, X)
                    dtype=cp.float32
                )
                
                # スケールファクター取得
                scale_chunk = scale_factors_gpu[t_start:t_end]
                
                # GPU上でベクトル化処理
                vol_chunk -= offset_value  # オフセット減算
                vol_chunk *= scale_chunk.reshape(-1, 1, 1, 1)  # 褪色補正
                vol_chunk = cp.clip(vol_chunk, 0, 65535)  # クリッピング
                
                # 型変換と結果保存
                vol_chunk = vol_chunk.astype(cp.uint16)
                result = cp.asnumpy(vol_chunk)
                dset_out[t_start:t_end, :, :, :] = result
                
                # メモリクリーンアップ
                del vol_chunk, result
                cp.cuda.Stream.null.synchronize()
                
                # 進捗情報表示
                if t_start % (chunk_size * 5) == 0:
                    elapsed = time.time() - start_time
                    processed_frames = t_end
                    fps = processed_frames / elapsed if elapsed > 0 else 0
                    remaining_frames = T_full - processed_frames
                    eta_minutes = (remaining_frames / fps / 60) if fps > 0 else 0
                    
                    print(f"  📊 進捗: {processed_frames}/{T_full} ({100*processed_frames/T_full:.1f}%)")
                    print(f"  🚀 速度: {fps:.1f}fps, VRAM: {memory_before:.1f}GB, ETA: {eta_minutes:.1f}分")
                    
            except cp.cuda.memory.OutOfMemoryError:
                print(f"💥 GPU メモリ不足 (chunk_size={chunk_size})")
                cp.get_default_memory_pool().free_all_blocks()
                continue
                
            except Exception as e:
                print(f"❌ 処理エラー (t={t_start}-{t_end}): {e}")
                continue
    
    # 最終メモリクリーンアップ
    try:
        del scale_factors_gpu
        cp.get_default_memory_pool().free_all_blocks()
        
    except Exception as cleanup_error:
        print(f"⚠️ メモリクリーンアップ中にエラー: {cleanup_error}")
        try:
            cp.get_default_memory_pool().free_all_blocks()
        except:
            pass
    
    # 処理結果サマリー
    total_elapsed = time.time() - start_time
    average_fps = T_full / total_elapsed if total_elapsed > 0 else 0
    
    print(f"\n🎉 全ボリューム単一チャンネル褪色補正GPU処理完了!")
    print(f"📊 処理フレーム数: {T_full}")
    print(f"🔄 チャンネル数: 1 (単一チャンネル)")
    print(f"⏱️  総処理時間: {total_elapsed/60:.1f}分")
    print(f"🚀 平均処理速度: {average_fps:.1f}フレーム/秒")
    print(f"💾 出力ファイル: {h5_output_path}")

# === 実行 ===
if GPU_AVAILABLE and scale_factors_all:
    print("🎯 単一チャンネル褪色補正GPU処理を開始します（全ボリューム）...")
    process_with_gpu_single_channel()
else:
    print("⚠️ GPU処理またはスケールファクターが利用できません")
    print(f"GPU利用可能: {GPU_AVAILABLE}")
    print(f"スケールファクター: {list(scale_factors_all.keys()) if scale_factors_all else 'なし'}")

Create MIP

In [ ]:
import h5py
import numpy as np
import tifffile as tiff
import os
from tqdm import tqdm

print(f"現在の時刻は {time.strftime('%Y-%m-%d %H:%M:%S')} です。")

# GPU可用性チェック
try:
    import cupy as cp
    GPU_AVAILABLE = True
    print("🚀 GPU処理が利用可能です")
    
    # GPU情報表示
    gpu_props = cp.cuda.runtime.getDeviceProperties(0)
    gpu_name = gpu_props['name'].decode('utf-8')
    total_memory_gb = cp.cuda.Device().mem_info[1] / (1024**3)
    print(f"🎮 GPU: {gpu_name}")
    print(f"💾 VRAM: {total_memory_gb:.1f}GB")
    
except ImportError:
    GPU_AVAILABLE = False
    print("⚠️ CPU処理を使用します")

# === パラメータ ===
calc_method = "_top90_mean"  # 使用する列のメソッド
hdf5_file = fr"I:\20240911_cam2_007_raw\20240911_cam2_007_raw_bleachcorrect{calc_method}_affine.h5"
output_path = r"I:\20240911_cam2_007_raw\orthogonal_views"
dirname = os.path.splitext(os.path.basename(hdf5_file))[0]
chunk_size = 100 if GPU_AVAILABLE else 50  # GPU使用時はより大きなチャンク

if not os.path.exists(output_path):
    os.makedirs(output_path)

# === HDF5ファイル情報取得 ===
with h5py.File(hdf5_file, 'r') as file:
    array = file['default']
    total_volumes, z, y, x = array.shape  # (T, Z, Y, X) - 単一チャンネル
    print(f"📐 データ形状: (T={total_volumes}, Z={z}, Y={y}, X={x})")
    print(f"🎯 処理対象: 全ボリューム（{total_volumes}フレーム）")

# === 出力配列の準備（全ボリューム・単一チャンネル） ===
orthogonal_view = np.zeros((total_volumes, y + z + 3, x + z + 3), dtype='uint16')  # チャンネル次元削除

def process_cpu_chunk(chunk_data, t_start, t_end):
    """CPU処理用のヘルパー関数（単一チャンネル対応）"""
    for i, t in enumerate(range(t_start, t_end)):
        data = chunk_data[i]  # (Z, Y, X) - チャンネルループ削除
        
        # CPU上で最大投影
        xy_proj = np.max(data, axis=0)  # (Y, X)
        yz_proj = np.max(data, axis=2).T  # (Z, Y)
        xz_proj = np.max(data, axis=1)  # (Z, X)
        
        # 直交ビューへの配置（チャンネル次元なし）
        orthogonal_view[t, 0:y, 0:x] = xy_proj
        orthogonal_view[t, 0:y, x+3:x+z+3] = yz_proj
        orthogonal_view[t, y+3:y+z+3, 0:x] = xz_proj
        orthogonal_view[t, y+z+2, x+z+2] = 1000

def process_orthogonal_projections():
    """直交投影処理（CPU/GPU自動選択・全ボリューム・単一チャンネル）"""
    use_gpu = GPU_AVAILABLE  # ローカル変数として処理状態を管理
    
    with h5py.File(hdf5_file, 'r') as file:
        array = file['default']
        
        for t_start in tqdm(range(0, total_volumes, chunk_size), 
                           desc="🎮 GPU直交投影（全ボリューム・単一チャンネル）" if use_gpu else "🖥️ CPU直交投影（全ボリューム・単一チャンネル）"):
            t_end = min(t_start + chunk_size, total_volumes)
            
            # チャンクデータ読み込み
            chunk_data = array[t_start:t_end]  # (chunk_size, Z, Y, X)
            
            if use_gpu:
                # GPU処理を試行
                try:
                    chunk_gpu = cp.asarray(chunk_data)
                    
                    for i, t in enumerate(range(t_start, t_end)):
                        data_gpu = chunk_gpu[i]  # (Z, Y, X) - チャンネルループ削除
                        
                        # GPU上で最大投影
                        xy_proj = cp.max(data_gpu, axis=0)  # (Y, X)
                        yz_proj = cp.max(data_gpu, axis=2).T  # (Z, Y)
                        xz_proj = cp.max(data_gpu, axis=1)  # (Z, X)
                        
                        # CPU側に結果をコピー（チャンネル次元なし）
                        orthogonal_view[t, 0:y, 0:x] = cp.asnumpy(xy_proj)
                        orthogonal_view[t, 0:y, x+3:x+z+3] = cp.asnumpy(yz_proj)
                        orthogonal_view[t, y+3:y+z+3, 0:x] = cp.asnumpy(xz_proj)
                        orthogonal_view[t, y+z+2, x+z+2] = 1000
                    
                    # GPUメモリクリーンアップ
                    del chunk_gpu
                    cp.get_default_memory_pool().free_all_blocks()
                    
                    # 進捗情報表示（GPU処理）
                    if t_start % (chunk_size * 5) == 0:
                        processed = t_end
                        remaining = total_volumes - processed
                        progress_pct = 100 * processed / total_volumes
                        print(f"  📊 GPU進捗: {processed}/{total_volumes} ({progress_pct:.1f}%)")
                    
                except Exception as e:
                    print(f"⚠️ GPU処理エラー、CPUにフォールバック: {e}")
                    use_gpu = False  # 今後のチャンクはCPU処理
                    # CPU処理で再試行
                    process_cpu_chunk(chunk_data, t_start, t_end)
                    
            else:
                # CPU処理
                process_cpu_chunk(chunk_data, t_start, t_end)
                
                # 進捗情報表示（CPU処理）
                if t_start % (chunk_size * 2) == 0:
                    processed = t_end
                    progress_pct = 100 * processed / total_volumes
                    print(f"  📊 CPU進捗: {processed}/{total_volumes} ({progress_pct:.1f}%)")

# === 処理実行 ===
print(f"🎯 全ボリューム（{total_volumes}フレーム）の単一チャンネル直交投影処理を開始...")
process_orthogonal_projections()
print("🎉 直交投影処理完了!")

# === TIFF保存（全ボリューム・単一チャンネル） ===
file_name = f"orthogonal_view_{dirname}_full_affine.tif"
output_path2 = os.path.join(output_path, file_name)

print("💾 TIFF保存中...")
print(f"📁 保存先: {output_path2}")
print(f"📊 データサイズ: {orthogonal_view.nbytes / (1024**3):.2f}GB")

try:
    tiff.imwrite(
        output_path2, 
        orthogonal_view, 
        imagej=True, 
        metadata={'axes': 'TYX'},  # チャンネル次元削除
        compression='zlib'
    )
    
    print(f"✅ 全ボリューム処理完了: {output_path2}")
    print(f"📈 出力形状: {orthogonal_view.shape}")
    print(f"📊 処理統計: {total_volumes}/{total_volumes}ボリューム (100%)")
    
except Exception as save_error:
    print(f"❌ TIFF保存エラー: {save_error}")
    print("💡 メモリ不足の可能性があります。チャンク保存を試します...")
    
    # === フォールバック: チャンク保存 ===
    save_chunk_size = 500  # 500フレームずつ保存
    
    for save_start in tqdm(range(0, total_volumes, save_chunk_size), desc="📦 チャンク保存"):
        save_end = min(save_start + save_chunk_size, total_volumes)
        chunk_name = f"orthogonal_view_{dirname}_part{save_start:05d}-{save_end:05d}.tif"
        chunk_path = os.path.join(output_path, chunk_name)
        
        chunk_data = orthogonal_view[save_start:save_end]
        tiff.imwrite(
            chunk_path,
            chunk_data,
            imagej=True,
            metadata={'axes': 'TYX'},  # チャンネル次元削除
            compression='zlib'
        )
        print(f"✅ 保存完了: {chunk_name}")
    
    print(f"📦 チャンク保存完了: {output_path}")

# === メモリクリーンアップ ===
try:
    del orthogonal_view
    if GPU_AVAILABLE:
        cp.get_default_memory_pool().free_all_blocks()
    print("🧹 メモリクリーンアップ完了")
except:
    pass